In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [2]:
# Read excel file Imputed_Table
df = pd.read_excel(r"C:\Users\s166927\Documents\Imputed_Table.xlsx")

In [3]:
df

,alim_grp_nom_eng,alim_ssgrp_nom_eng,alim_ssssgrp_nom_eng,alim_nom_eng,Energy..Regulation.EU.No.1169.2011..kcal.100g.,Water..g.100g.,Protein..g.100g.,Carbohydrate..g.100g.,Fat..g.100g.,FA.saturated..g.100g.,...,Vitamin.B2.or.Riboflavin..mg.100g.,Vitamin.B3.or.Niacin..mg.100g.,Vitamin.B5.or.Pantothenic.acid..mg.100g.,Vitamin.B6..mg.100g.,Vitamin.B9.or.Folate..µg.100g.,Vitamin.B12..µg.100g.,Fibres..g.100g.,Salt..g.100g.,Sugars..g.100g.,Cholesterol..mg.100g.
0,NaN,NaN,NaN,Desert (average),NaN,45.4,4.63,36.60,12.90,5.18,...,0.150,0.61,0.40,0.056,30.80,0.210,1.54,0.380,23.7000,56.70
1,starters and dishes,mixed salads,-,"Prepared mixed tuna and vegetable salad, canned",128.404,76.5,9.15,7.74,4.70,0.56,...,0.053,4.45,0.08,0.290,31.00,1.450,2.70,1.110,3.0800,19.20
2,starters and dishes,mixed salads,-,"Prepared mixed meat/fish canned, salad",134.835,76.7,8.06,6.40,5.30,0.16,...,0.022,4.13,0.20,0.120,11.10,1.230,2.00,0.950,1.9000,15.20
3,starters and dishes,mixed salads,-,"Greek-style marinated mushrooms, prepacked",112.270,85.2,2.08,3.95,3.55,0.23,...,0.210,1.84,0.88,0.088,19.60,0.018,2.35,1.260,2.3800,0.11
4,starters and dishes,mixed salads,-,"Prepared potatoes salad, home-made",127.313,76.0,2.68,9.90,8.20,1.43,...,0.060,0.89,0.53,0.140,7.00,0.000,1.30,1.320,1.9903,68.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3181,baby food,baby deserts,-,"Dairy dessert for baby, plain with sugar or wi...",100.000,77.7,3.19,12.50,3.50,2.28,...,0.110,0.12,0.26,0.029,9.06,0.200,1.50,0.075,9.2600,11.00
3182,baby food,baby biscuits and cereals,-,Instant cereal (powder to be reconstituted) fo...,393.000,2.2,5.10,88.30,1.70,0.64,...,0.040,5.02,1.33,0.380,100.00,0.440,1.80,0.041,12.7000,350.00
3183,baby food,baby biscuits and cereals,-,Instant cereal (powder to be reconstituted) fo...,391.000,1.8,10.00,81.10,2.00,0.61,...,0.040,4.43,1.58,0.290,86.50,0.290,4.30,0.037,32.8000,1.72
3184,baby food,baby biscuits and cereals,-,Biscuit for baby,439.000,3.0,7.50,73.30,12.30,4.35,...,0.620,5.90,4.69,0.660,31.10,0.280,2.40,0.360,24.8000,23.50


In [4]:
# Rename columns to more readable names
df.rename({'alim_grp_nom_eng': 'Food Group', 'alim_ssgrp_nom_eng': 'Food Subgroup',
 'alim_ssssgrp_nom_eng': 'Food Subsubgroup', 'alim_nom_eng': 'Food Name',
 'Energy..Regulation.EU.No.1169.2011..kcal.100g.': 'Energy',
 'Water..g.100g.': 'Water', 'Protein..g.100g.': 'Protein', 'Carbohydrate..g.100g.': 'Carbohydrate',
 'Fat..g.100g.': 'Fat', 'FA.saturated..g.100g.': 'FA Saturated',
 'FA.mono..g.100g.': 'FA Mono', 'FA.poly..g.100g.': 'FA Poly',
 'Calcium..mg.100g.': 'Calcium', 'Chloride..mg.100g.': 'Chloride',
 'Copper..mg.100g.': 'Copper', 'Iron..mg.100g.': 'Iron',
 'Iodine..µg.100g.': 'Iodine', 'Magnesium..mg.100g.': 'Magnesium',
 'Manganese..mg.100g.': 'Manganese', 'Phosphorus..mg.100g.': 'Phosphorus',
 'Potassium..mg.100g.': 'Potassium', 'Selenium..µg.100g.': 'Selenium',
 'Sodium..mg.100g.': 'Sodium', 'Zinc..mg.100g.': 'Zinc',
 'Vitamin.A..µg.100g.': 'Vitamin A', 'Vitamin.D..µg.100g.': 'Vitamin D',
 'Vitamin.E..mg.100g.': 'Vitamin E', 'Vitamin.K..µg.100g.': 'Vitamin K',
 'Vitamin.C..mg.100g.': 'Vitamin C', 'Vitamin.B1.or.Thiamin..mg.100g.': 'Vitamin B1',
 'Vitamin.B2.or.Riboflavin..mg.100g.': 'Vitamin B2', 'Vitamin.B3.or.Niacin..mg.100g.': 'Vitamin B3',
 'Vitamin.B5.or.Pantothenic.acid..mg.100g.': 'Vitamin B5', 'Vitamin.B6..mg.100g.': 'Vitamin B6',
 'Vitamin.B9.or.Folate..µg.100g.': 'Vitamin B9', 'Vitamin.B12..µg.100g.': 'Vitamin B12',
 'Fibres..g.100g.': 'Fibres', 'Salt..g.100g.': 'Salt',
 'Sugars..g.100g.': 'Sugar', 'Cholesterol..mg.100g.': 'Cholesterol'}, axis = 1, inplace = True)

In [5]:
df.head(5)

,Food Group,Food Subgroup,Food Subsubgroup,Food Name,Energy,Water,Protein,Carbohydrate,Fat,FA Saturated,...,Vitamin B2,Vitamin B3,Vitamin B5,Vitamin B6,Vitamin B9,Vitamin B12,Fibres,Salt,Sugar,Cholesterol
0,NaN,NaN,NaN,Desert (average),NaN,45.4,4.63,36.60,12.90,5.18,...,0.150,0.61,0.40,0.056,30.8,0.210,1.54,0.38,23.7000,56.70
1,starters and dishes,mixed salads,-,"Prepared mixed tuna and vegetable salad, canned",128.404,76.5,9.15,7.74,4.70,0.56,...,0.053,4.45,0.08,0.290,31.0,1.450,2.70,1.11,3.0800,19.20
2,starters and dishes,mixed salads,-,"Prepared mixed meat/fish canned, salad",134.835,76.7,8.06,6.40,5.30,0.16,...,0.022,4.13,0.20,0.120,11.1,1.230,2.00,0.95,1.9000,15.20
3,starters and dishes,mixed salads,-,"Greek-style marinated mushrooms, prepacked",112.270,85.2,2.08,3.95,3.55,0.23,...,0.210,1.84,0.88,0.088,19.6,0.018,2.35,1.26,2.3800,0.11
4,starters and dishes,mixed salads,-,"Prepared potatoes salad, home-made",127.313,76.0,2.68,9.90,8.20,1.43,...,0.060,0.89,0.53,0.140,7.0,0.000,1.30,1.32,1.9903,68.00


In [6]:
# Remove food Desert (average)
df = df.drop(df.index[df['Food Name'] == 'Desert (average)'])

In [7]:
# Search for duplicates in column Food Name
df.loc[df.duplicated(['Food Name'])]

,Food Group,Food Subgroup,Food Subsubgroup,Food Name,Energy,Water,Protein,Carbohydrate,Fat,FA Saturated,...,Vitamin B2,Vitamin B3,Vitamin B5,Vitamin B6,Vitamin B9,Vitamin B12,Fibres,Salt,Sugar,Cholesterol
3146,miscellaneous,miscellaneous ingredients for vegetarian,-,Wheat bran,299.93,14.9355,11.3822,44.936,3.834,1.38176,...,0.492071,17.4536,5.6941,0.863777,259.3932,13.5372,8.6798,0.702885,14.0094,0.049


In [8]:
df.index[df['Food Name'] == 'Wheat bran']
# Other index is 3120

Int64Index([3120, 3146], dtype='int64')

In [9]:
# Check other wheat bran
df.iloc[3119:3120]

,Food Group,Food Subgroup,Food Subsubgroup,Food Name,Energy,Water,Protein,Carbohydrate,Fat,FA Saturated,...,Vitamin B2,Vitamin B3,Vitamin B5,Vitamin B6,Vitamin B9,Vitamin B12,Fibres,Salt,Sugar,Cholesterol
3120,miscellaneous,miscellaneous ingredients,-,Wheat bran,279.0,9.05,15.4,23.6,4.35,0.84,...,0.47,21.6,2.29,1.34,110.0,0.0,42.0,0.033,2.46,0.0


In [10]:
# Delete duplicate wheat bran on index 3145
df.drop(df.index[3145], axis = 0, inplace = True)
# Reset the row index to fill the gaps left behind by the removed rows
df = df.reset_index(drop = True)

In [11]:
# Create list of lists from dataframe df
tables = df.values.tolist()

In [12]:
# Define sets nutrients, foods and food groups
nutrients = df.columns.tolist()[4:40] # Create list with all nutrients
foods = list(df['Food Name']) # Create list with all food names
food_group = list(df['Food Group'].unique()) # get every unique food group
food_subgroup = list(df['Food Subgroup'].unique()) # get every unique food subgroup
food_subsubgroup = list(df['Food Subsubgroup'].unique()) # get every unique food sub subgroup

In [425]:
# Define parameters
# fat and fatty acids depend on energy amount in the diet plan, so their bounds
# can be calculated after creating a diet plan.
# RDA water is 3.020 liters, but data is in g/100g, so constraint of water
# is converted from liter to g, so lower bound is 3020 g and upper bound is 6000 g.
energy = 2000 # a starter value is needed, otherwise we get an error
# The variable energy only appears in the function NutrientValues and in
# this function the current energy amount is taken to calculate the lower
# and upper bounds for fat and the fatty acids
inf = GRB.INFINITY # define infinity to variable 'inf', because some nutrients have no upper bound.
Lower_bound = {'Energy': 1500, 'Water': 3020, 'Protein': 62.8,
 'Carbohydrate': 279.11, 'Fat': energy*0.2/9, 'Saturated Fat': 0,
 'FA Mono': energy*0.1/9, 'FA Poly': energy*0.03/9, 'Calcium': 930,
 'Chloride': 2057.14, 'Copper': 1.11, 'Iron': 9.87, 'Iodine': 154,
 'Magnesium': 354, 'Manganese': 3.26, 'Phosphorus': 725, 'Potassium': 3580,
 'Selenium': 63.5, 'Sodium': 1702.22, 'Zinc': 10.9, 'Vitamin A': 815,
 'Vitamin D': 11.75, 'Vitamin E': 11.9, 'Vitamin K': 92.9, 'Vitamin C': 82,
 'Vitamin B1': 1.2, 'Vitamin B2': 1.415, 'Vitamin B3': 16.45, 
 'Vitamin B5': 5.31, 'Vitamin B6': 1.51, 'Vitamin B9': 315, 'Vitamin B12': 2.64,
 'Fibres': 30.16, 'Salt': 0, 'Sugar': 0, 'Cholesterol': 0}
Upper_bound = {'Energy': 2874.5, 'Water': 6000, 'Protein': 250,
 'Carbohydrate': 315, 'Fat': energy*0.35/9, 'Saturated Fat': energy*0.1/9,
 'FA Mono': energy*0.2/9, 'FA Poly': energy*0.12/9, 'Calcium': 2500,
 'Chloride': 3600, 'Copper': 5, 'Iron': 45, 'Iodine': 600, 'Magnesium': 704,
 'Manganese': 11, 'Phosphorus': 4000, 'Potassium': inf, 'Selenium': 300,
 'Sodium': 2300, 'Zinc': 25, 'Vitamin A': 3000, 'Vitamin D': 100,
 'Vitamin E': 300, 'Vitamin K': inf, 'Vitamin C': 2000, 'Vitamin B1': inf,
 'Vitamin B2': inf, 'Vitamin B3': 900, 'Vitamin B5': 200, 'Vitamin B6': 25,
 'Vitamin B9': 1000, 'Vitamin B12': inf, 'Fibres': inf, 'Salt': 6,
 'Sugar': 50, 'Cholesterol': 300}

In [30]:
# Create function that prints objective value and the diet plan
def DietPlanSolution():
    if model.status == GRB.OPTIMAL: # if model has a solution/is feasible
        # print objective value
        print('\nObjective Value:')
        # objective function can change when creating a new model
        # for example, create a diet that minimizes amount of foods
        print('\nEnergy: %g' % model.ObjVal)
        print('\nDiet Plan:')
        for i in range(len(foods)): # loop over every food
            if x[i].x > 0.00001: # if food is in diet plan
                # print food name and amount in the diet plan and multiply by 100
                # because ciqual tables provide every nutrient value per 100g
                print(tables[i][3], x[i].x*100)

In [31]:
# Create function that calculates amount of each nutrient based on diet plan
def NutrientValues():
    nutrient_value_list = [0]*36 # create empty list
    if model.status == GRB.OPTIMAL: # if model is feasible
        for i in range(len(foods)): # loop over every food
            if x[i].x > 0.00001: # if food is in diet plan
                for j in range(len(nutrients)): # then loop over nutrients
                    # multiply every nutrient value by amount in diet and add it to the list
                    nutrient_value_list[j] += df.iloc[i,j+4]*x[i].x
    
    energy = nutrient_value_list[0] # Energy amount in diet plan
    # calculate lower bounds for fat and FA mono/poly
    Lower_bound["Fat"] = energy*0.2/9
    Lower_bound["FA Mono"] = energy*0.1/9
    Lower_bound["FA Poly"] = energy*0.03/9
    lower_bound_list = Lower_bound.values() # create list of lower bound values

    # calculate upper bounds for fat, saturated fat and FA mono/poly
    Upper_bound["Fat"] = energy*0.35/9
    Upper_bound["Saturated Fat"] = energy*0.1/9
    Upper_bound["FA Mono"] = energy*0.2/9
    Upper_bound["FA Poly"] = energy*0.12/9
    upper_bound_list = Upper_bound.values() # create list of upper bound values
    # w = nutrient, x = lb, y = amount in diet plan, z = upper bound
    nutrient_amount = [[w,x,y,z] for w, x, y, z in zip(nutrients, lower_bound_list, nutrient_value_list, upper_bound_list)]
    print(*nutrient_amount, sep = "\n") # every list is printed on a new line


In [32]:
# Create function that calculates the percentage of nutrients gained from amount of food in diet plan
def NutrientPercentage():
    # nutrient_value_list has to be computed again
    nutrient_value_list = [0]*36 # create empty list
    if model.status == GRB.OPTIMAL: # if model is feasible
        for i in range(len(foods)): # loop over every food
            if x[i].x > 0.00001: # if food is in diet plan
                for j in range(len(nutrients)): # then loop over nutrients
                    # multiply every nutrient value by amount in diet and add it to the list
                    nutrient_value_list[j] += df.iloc[i,j+4]*x[i].x
    
    # with this you can check how foods contribute to nutrients
    for j in range(len(nutrients)):
        if nutrient_value_list[j] != 0:
            for i in range(len(foods)): # loop over every food
                if x[i].x > 0: # if food is in diet plan
                    print(int(df.iloc[i,j+4]*x[i].x/nutrient_value_list[j]*100), end = " ")
            print(nutrients[j]) # print nutrient at end of row
            

In [473]:
# reset the model, so you can start over
model.reset()

Discarded solution information


In [521]:
# Create model
model = gp.Model('Diet Problem Basic')

In [522]:
# Define decision variables x and y
# x defines the amount of food in the diet plan
# y is a binary variable that shows whether or not a food is in the diet plan
# By default, the lower bound of x is 0
x = [model.addVar(vtype = GRB.CONTINUOUS, name = "x") for i in range(len(tables))]
y = [model.addVar(vtype = GRB.BINARY, name = "y") for i in range(len(tables))]

In [523]:
# Minimize energy
model.setObjective(gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))), GRB.MINIMIZE)

In [524]:
# Create big M constraint.
# Big M constraint is used to 
model.addConstrs(x[i] <= 100*y[i] for i in range(len(tables))) # Big M constraint

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [525]:
# Create constraints for fatty acids
# fat
model.addConstr(gp.quicksum(x[i]*tables[i][8] for i in range(len(tables))) <= 0.35/9*gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))))
model.addConstr(gp.quicksum(x[i]*tables[i][8] for i in range(len(tables))) >= 0.2/9*gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))))
# saturated fat
model.addConstr(gp.quicksum(x[i]*tables[i][9] for i in range(len(tables))) <= 0.1/9*gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))))
# FA mono
model.addConstr(gp.quicksum(x[i]*tables[i][10] for i in range(len(tables))) <= 0.2/9*gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))))
model.addConstr(gp.quicksum(x[i]*tables[i][10] for i in range(len(tables))) >= 0.1/9*gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))))
# FA poly
model.addConstr(gp.quicksum(x[i]*tables[i][11] for i in range(len(tables))) <= 0.12/9*gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))))
model.addConstr(gp.quicksum(x[i]*tables[i][11] for i in range(len(tables))) >= 0.03/9*gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


<gurobi.Constr *Awaiting Model Update*>

In [526]:
# Create constraints for nutrients
model.addConstr(gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))) <= 2874.5) # energy
model.addConstr(gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))) >= 1500)
model.addConstr(gp.quicksum(x[i]*tables[i][5] for i in range(len(tables))) <= 6000) # water
model.addConstr(gp.quicksum(x[i]*tables[i][5] for i in range(len(tables))) >= 3020)
model.addConstr(gp.quicksum(x[i]*tables[i][6] for i in range(len(tables))) <= 250) # protein
model.addConstr(gp.quicksum(x[i]*tables[i][6] for i in range(len(tables))) >= 62.8)
model.addConstr(gp.quicksum(x[i]*tables[i][7] for i in range(len(tables))) <= 315) # carbohydrate
model.addConstr(gp.quicksum(x[i]*tables[i][7] for i in range(len(tables))) >= 279.11)
model.addConstr(gp.quicksum(x[i]*tables[i][12] for i in range(len(tables))) <= 2500) # calcium
model.addConstr(gp.quicksum(x[i]*tables[i][12] for i in range(len(tables))) >= 930)
model.addConstr(gp.quicksum(x[i]*tables[i][13] for i in range(len(tables))) <= 3600) # chloride
model.addConstr(gp.quicksum(x[i]*tables[i][13] for i in range(len(tables))) >= 2057.14)
model.addConstr(gp.quicksum(x[i]*tables[i][14] for i in range(len(tables))) <= 5) # copper
model.addConstr(gp.quicksum(x[i]*tables[i][14] for i in range(len(tables))) >= 1.11)
model.addConstr(gp.quicksum(x[i]*tables[i][15] for i in range(len(tables))) <= 45) # iron
model.addConstr(gp.quicksum(x[i]*tables[i][15] for i in range(len(tables))) >= 9.87)
model.addConstr(gp.quicksum(x[i]*tables[i][16] for i in range(len(tables))) <= 600) # iodine
model.addConstr(gp.quicksum(x[i]*tables[i][16] for i in range(len(tables))) >= 154)
model.addConstr(gp.quicksum(x[i]*tables[i][17] for i in range(len(tables))) <= 704) # magnesium
model.addConstr(gp.quicksum(x[i]*tables[i][17] for i in range(len(tables))) >= 354)
model.addConstr(gp.quicksum(x[i]*tables[i][18] for i in range(len(tables))) <= 11) # manganese
model.addConstr(gp.quicksum(x[i]*tables[i][18] for i in range(len(tables))) >= 3.26)
model.addConstr(gp.quicksum(x[i]*tables[i][19] for i in range(len(tables))) <= 4000) # phosphorus
model.addConstr(gp.quicksum(x[i]*tables[i][19] for i in range(len(tables))) >= 725)
model.addConstr(gp.quicksum(x[i]*tables[i][20] for i in range(len(tables))) <= inf) # potassium
model.addConstr(gp.quicksum(x[i]*tables[i][20] for i in range(len(tables))) >= 3580)
model.addConstr(gp.quicksum(x[i]*tables[i][21] for i in range(len(tables))) <= 300) # selenium
model.addConstr(gp.quicksum(x[i]*tables[i][21] for i in range(len(tables))) >= 63.5)
model.addConstr(gp.quicksum(x[i]*tables[i][22] for i in range(len(tables))) <= 2300) # sodium
model.addConstr(gp.quicksum(x[i]*tables[i][22] for i in range(len(tables))) >= 1702.22)
model.addConstr(gp.quicksum(x[i]*tables[i][23] for i in range(len(tables))) <= 25) # zinc
model.addConstr(gp.quicksum(x[i]*tables[i][23] for i in range(len(tables))) >= 10.9)
model.addConstr(gp.quicksum(x[i]*tables[i][24] for i in range(len(tables))) <= 3000) # vitamin a
model.addConstr(gp.quicksum(x[i]*tables[i][24] for i in range(len(tables))) >= 815)
model.addConstr(gp.quicksum(x[i]*tables[i][25] for i in range(len(tables))) <= 100) # vitamin d
model.addConstr(gp.quicksum(x[i]*tables[i][25] for i in range(len(tables))) >= 11.75)
model.addConstr(gp.quicksum(x[i]*tables[i][26] for i in range(len(tables))) <= 300) # vitamin e
model.addConstr(gp.quicksum(x[i]*tables[i][26] for i in range(len(tables))) >= 11.9)
model.addConstr(gp.quicksum(x[i]*tables[i][27] for i in range(len(tables))) <= inf) # vitamin k
model.addConstr(gp.quicksum(x[i]*tables[i][27] for i in range(len(tables))) >= 92.9)
model.addConstr(gp.quicksum(x[i]*tables[i][28] for i in range(len(tables))) <= 2000) # vitamin c
model.addConstr(gp.quicksum(x[i]*tables[i][28] for i in range(len(tables))) >= 82)
model.addConstr(gp.quicksum(x[i]*tables[i][29] for i in range(len(tables))) <= inf) # vitamin b1
model.addConstr(gp.quicksum(x[i]*tables[i][29] for i in range(len(tables))) >= 1.2)
model.addConstr(gp.quicksum(x[i]*tables[i][30] for i in range(len(tables))) <= inf) # vitamin b2
model.addConstr(gp.quicksum(x[i]*tables[i][30] for i in range(len(tables))) >= 1.415)
model.addConstr(gp.quicksum(x[i]*tables[i][31] for i in range(len(tables))) <= 900) # vitamin b3
model.addConstr(gp.quicksum(x[i]*tables[i][31] for i in range(len(tables))) >= 16.45)
model.addConstr(gp.quicksum(x[i]*tables[i][32] for i in range(len(tables))) <= 200) # vitamin b5
model.addConstr(gp.quicksum(x[i]*tables[i][32] for i in range(len(tables))) >= 5.31)
model.addConstr(gp.quicksum(x[i]*tables[i][33] for i in range(len(tables))) <= 25) # vitamin b6
model.addConstr(gp.quicksum(x[i]*tables[i][33] for i in range(len(tables))) >= 1.51)
model.addConstr(gp.quicksum(x[i]*tables[i][34] for i in range(len(tables))) <= 1000) # vitamin b9
model.addConstr(gp.quicksum(x[i]*tables[i][34] for i in range(len(tables))) >= 315)
model.addConstr(gp.quicksum(x[i]*tables[i][35] for i in range(len(tables))) <= inf) # vitamin b12
model.addConstr(gp.quicksum(x[i]*tables[i][35] for i in range(len(tables))) >= 2.64)
model.addConstr(gp.quicksum(x[i]*tables[i][36] for i in range(len(tables))) <= inf) # fibres
model.addConstr(gp.quicksum(x[i]*tables[i][36] for i in range(len(tables))) >= 30.16)
model.addConstr(gp.quicksum(x[i]*tables[i][37] for i in range(len(tables))) <= 6) # salt
model.addConstr(gp.quicksum(x[i]*tables[i][37] for i in range(len(tables))) >= 0)
model.addConstr(gp.quicksum(x[i]*tables[i][38] for i in range(len(tables))) <= 50) # sugar
model.addConstr(gp.quicksum(x[i]*tables[i][38] for i in range(len(tables))) >= 0)
model.addConstr(gp.quicksum(x[i]*tables[i][39] for i in range(len(tables))) <= 300) # cholesterol
model.addConstr(gp.quicksum(x[i]*tables[i][39] for i in range(len(tables))) >= 0)

<gurobi.Constr *Awaiting Model Update*>

In [527]:
model.optimize() # optimize the model
DietPlanSolution() # print the diet plan using the function DietPlanSolution

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3255 rows, 6368 columns and 213972 nonzeros
Model fingerprint: 0xb2640317
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]
Presolve removed 3194 rows and 3191 columns
Presolve time: 0.55s
Presolved: 61 rows, 3177 columns, 179894 nonzeros
Variable types: 3177 continuous, 0 integer (0 binary)

Root relaxation: objective 1.500000e+03, 45 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1500.0000000 1500.00000  0.00%     -    0s

Explored 0 nodes (45 simplex iterations) in 0.73 seconds
Thread count was 8 (of 8 avai

In [528]:
# Calculate the amount of each nutrient based on the diet plan using the function NutrientValues
NutrientValues()

['Energy', 1500, 1499.9999999999995, 2874.5]
['Water', 3020, 5862.414801807505, 6000]
['Protein', 62.8, 62.80000000000001, 250]
['Carbohydrate', 279.11, 279.10999999999996, 315]
['Fat', 33.33333333333333, 39.770688110147624, 58.33333333333331]
['FA Saturated', 0, 9.286939481293437, 16.666666666666664]
['FA Mono', 16.666666666666664, 16.666666666666604, 33.33333333333333]
['FA Poly', 4.999999999999998, 9.883635433925999, 19.999999999999993]
['Calcium', 930, 2500.000000000001, 2500]
['Chloride', 2057.14, 3600.000000000002, 3600]
['Copper', 1.11, 2.5806738715972157, 5]
['Iron', 9.87, 20.109922579604792, 45]
['Iodine', 154, 600.0000000000002, 600]
['Magnesium', 354, 406.43819309260647, 704]
['Manganese', 3.26, 11.000000000000002, 11]
['Phosphorus', 725, 1300.298066497518, 4000]
['Potassium', 3580, 3580.000000000001, 1e+100]
['Selenium', 63.5, 300.0000000000007, 300]
['Sodium', 1702.22, 2300.0, 2300]
['Zinc', 10.9, 10.899999999999999, 25]
['Vitamin A', 815, 3000.0000000000005, 3000]
['Vitam

In [529]:
# Calculate the percentage of nutrients gained from a food in the diet plan with the function NutrientPercentage
NutrientPercentage()

4 40 4 1 10 0 0 1 2 5 0 0 6 12 3 1 1 0 0 1 0 1 Energy
0 0 1 1 1 0 0 0 0 0 0 90 0 0 0 0 1 0 0 0 0 0 Water
1 0 0 0 52 0 0 3 14 17 0 0 1 0 0 0 2 0 0 1 1 0 Protein
8 53 5 1 0 0 0 0 0 0 0 0 6 20 0 0 2 0 0 0 0 0 Carbohydrate
0 0 0 0 5 0 0 1 0 10 0 0 1 0 13 7 52 0 0 0 0 0 Fat
1 0 0 0 11 0 0 2 0 7 0 1 2 0 35 5 29 0 0 0 0 0 FA Saturated
0 0 0 0 5 0 0 0 0 7 0 0 0 0 10 2 69 0 0 0 0 0 FA Mono
0 0 0 0 0 0 0 0 0 9 0 2 4 0 1 19 57 0 0 0 0 0 FA Poly
0 1 0 1 0 0 0 0 0 1 0 0 1 66 0 0 1 0 0 0 0 25 Calcium
2 6 0 0 2 0 0 1 2 2 0 44 1 0 0 0 35 0 0 0 0 0 Chloride
2 1 2 2 3 0 0 41 1 0 0 37 0 2 0 0 1 0 0 0 0 0 Copper
2 13 0 1 13 0 0 21 0 1 0 3 8 23 0 0 3 0 0 1 0 3 Iron
0 0 2 0 1 0 0 0 3 0 0 1 0 0 0 0 2 85 0 0 0 0 Iodine
7 0 1 2 8 0 0 0 3 4 0 26 4 34 0 0 1 0 0 1 0 0 Magnesium
0 1 0 2 0 0 0 0 0 0 0 92 0 1 0 0 1 0 0 0 0 0 Manganese
2 0 1 1 18 0 0 2 6 9 0 4 5 2 0 0 3 0 0 0 1 39 Phosphorus
11 0 4 3 12 0 0 0 5 5 0 37 13 0 0 0 2 0 0 0 0 0 Potassium
0 0 4 0 1 0 0 2 74 2 0 0 0 4 0 0 4 0 0 0 0 0 Selenium
0 0 0 0 3 0 0 0

In [530]:
# Add constraints to the model
# tables[i][0] refers to the column 'Food Group'
# tables[i][1] refers to the column 'Food Subgroup'
# tables[i][2] refers to the column 'Food Subsubgroup'
# tables[i][3] refers to the column 'Food Name'
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "seafood, raw") <= 0) # Create a constraint which excludes a food subgroup from the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "beverages") <= 24) # Create a constraint that puts an upper bound on a food group
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tapioca, raw") <= 0) # Constraint that excludes a specific food from the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "offals") <= 0) # Create a constraint which excludes a food subsubgroup from the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "coffee, tea, cocoa beverages, etc. ready to drink") <= 10)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Baking powder or raising agent") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "fish, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Baker's yeast, dehydrated") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Baker's yeast, compressed") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "meat, egg and fish") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "fats and oils") <= 1) # Create an upper bound for the number of fats and oils in the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "sauces") <= 0.5)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "vegetables from Martinique and their products") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "fruits from Martinique and their products") <= 0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "meat, egg and fish") <= 4)
for i in range(len(foods)):
    if tables[i][0] == "meat, egg and fish": # If a food from the food group 'meat, egg and fish' is present in the diet plan,
        model.addConstr(x[i] >= 0.5*y[i]) # Then the amount of the food has to be at least 50 g to avoid having small portions
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "seaweed") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "seaweed":
        model.addConstr(x[i] >= 0.05*y[i])

In [531]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4076 rows, 6368 columns and 217905 nonzeros
Model fingerprint: 0x6bf28f45
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 6e+03]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R3265 by 74.000000000

Presolve removed 2749 rows and 2814 columns
Presolve time: 0.67s
Presolved: 1327 rows, 3554 columns, 167303 nonzeros
Variable types: 2888 continuous, 666 integer (666 binary)

Root relaxation: objective 1.500000e+03, 36 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

In [532]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "vegetables") <= 5.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "vegetables") <= 5)
for i in range(len(foods)):
    if tables[i][1] == "vegetables":
        model.addConstr(x[i] >= 1.0*y[i])
for i in range(len(foods)):
    if tables[i][1] == "vegetables":
        model.addConstr(x[i] <= 2.5*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "fruits") <= 3.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "fruits") <= 3)
for i in range(len(foods)):
    if tables[i][1] == "fruits":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "nuts and seeds") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "nuts and seeds":
        model.addConstr(x[i] >= 0.30*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "baby food") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "potatoes and other tubers") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "potatoes and other tubers") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "potatoes and other tubers":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "legumes") <= 2.0)        
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "legumes") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "legumes":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "breakfast cereals") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "breakfast cereals":
        model.addConstr(x[i] >= 0.5*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "cooking aids") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "condiments") <= 0.30)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "foods for particular nutritional uses") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "cooking ingredients") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "alcoholic beverages") <= 5)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "beverages") <= 3)
for i in range(len(foods)):
    if tables[i][0] == "beverages":
        model.addConstr(x[i] >= 1.5*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "sweet biscuits") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "sweet biscuits":
        model.addConstr(x[i] >= 0.1*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "pasta, rice and grains") <= 1.50)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "pasta, rice and grains, raw") <= 0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "pasta, rice and grains") <= 2)
for i in range(len(foods)):
    if tables[i][1] == "pasta, rice and grains":
        model.addConstr(x[i] >= 0.75*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "savoury biscuits") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "savoury biscuits":
        model.addConstr(x[i] >= 0.50*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "starters and dishes") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "starters and dishes") <= 2)
for i in range(len(foods)):
    if tables[i][0] == "starters and dishes":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "milk") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "milk":
        model.addConstr(x[i] >= 1.5*y[i])        
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "dairy products and similar") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "dairy products and similar") <= 3)
for i in range(len(foods)):
    if tables[i][1] == "dairy products and similar":
        model.addConstr(x[i] >= 0.50*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "fromages blanc" or tables[i][1] == "cheese and similar") <= 0.60)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "cheese and similar") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "cheese and similar":
        model.addConstr(x[i] >= 0.30*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "herbs" or tables[i][1] == "spices") <= 0.3)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "herbs" or tables[i][1] == "spices") <= 2)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "raw meat") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "fats and oils") <= 0.4)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "sauces") <= 2)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "breads and similar") <= 2)
for i in range(len(foods)):
    if tables[i][1] == "breads and similar":
        model.addConstr(x[i] >= 0.4*y[i])

In [533]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6210 rows, 6368 columns and 225796 nonzeros
Model fingerprint: 0xc3cb675e
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 6e+03]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R4077 by 298.000000000

Presolve removed 1675 rows and 1449 columns
Presolve time: 0.58s
Presolved: 4535 rows, 4919 columns, 157491 nonzeros
Variable types: 2599 continuous, 2320 integer (2320 binary)

Root relaxation: objective 1.500000e+03, 40 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [534]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Basil, fresh") <= 0.03)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Lettuce, sucrine, raw" or tables[i][3] == "Lettuce, oak-leaf, raw") <= 2.50)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Aspartame sweetener, in tablets" or tables[i][3] == "Stevia sweetener" 
                            or tables[i][3] == "Aspartame, in powder" or tables[i][3] == "Saccharin sweetener") <= 0.0032)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Lime, pulp, raw") <= 1.0)

<gurobi.Constr *Awaiting Model Update*>

In [535]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6214 rows, 6368 columns and 225804 nonzeros
Model fingerprint: 0x00cdff42
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve produced solution with objective 2092.04 (0.09s)
Loaded MIP start from previous solve with objective 2092.04

Presolve removed 1679 rows and 1450 columns
Presolve time: 0.57s
Presolved: 4535 rows, 4918 columns, 157493 nonzeros
Variable types: 2598 continuous, 2320 integer (2320 binary)

Root relaxation: objective 1.500000e+03, 145 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0

In [536]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chinese or Japanese artichokes, frozen, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cod liver oil") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Morel, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Lemon zest, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chewing gum, without sugar") <= 0.20)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "cream sauce for pizza base") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [537]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6220 rows, 6368 columns and 225810 nonzeros
Model fingerprint: 0x10bf8331
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1692 rows and 1459 columns
Presolve time: 0.57s
Presolved: 4528 rows, 4909 columns, 157207 nonzeros
Variable types: 2593 continuous, 2316 integer (2316 binary)

Root relaxation: objective 1.500000e+03, 81 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    7          - 1500.00000      -     -    0s
H    0

In [538]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tomato sauce for pizza base") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Oat bran") <= 0.15)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Pumpkin, cooked") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [539]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6223 rows, 6368 columns and 225813 nonzeros
Model fingerprint: 0x4eb6010e
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1697 rows and 1462 columns
Presolve time: 0.58s
Presolved: 4526 rows, 4906 columns, 157082 nonzeros
Variable types: 2591 continuous, 2315 integer (2315 binary)

Root relaxation: objective 1.500000e+03, 70 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    8          - 1500.00000      -     -    0s
     0

In [540]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Rabbit pate") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Nutritional yeast") <= 0.3)

<gurobi.Constr *Awaiting Model Update*>

In [541]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6225 rows, 6368 columns and 225815 nonzeros
Model fingerprint: 0x9828f187
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1701 rows and 1464 columns
Presolve time: 0.59s
Presolved: 4524 rows, 4904 columns, 157017 nonzeros
Variable types: 2590 continuous, 2314 integer (2314 binary)

Root relaxation: objective 1.500000e+03, 88 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    6          - 1500.00000      -     -    0s
     0

In [542]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cloves") <= 0.05)

<gurobi.Constr *Awaiting Model Update*>

In [543]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6226 rows, 6368 columns and 225816 nonzeros
Model fingerprint: 0x0257b886
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve produced solution with objective 1500 (0.10s)
Loaded MIP start from previous solve with objective 1500

Presolve removed 1702 rows and 1464 columns
Presolve time: 0.60s
Presolved: 4524 rows, 4904 columns, 157017 nonzeros
Variable types: 2590 continuous, 2314 integer (2314 binary)

Root relaxation: cutoff, 18 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1

In [544]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Rice bran") <= 0.15)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Gelatine, dried") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Wheat germ") <= 0.1)
for i in range(len(foods)):
    if tables[i][1] == "chocolate and chocolate products":
        model.addConstr(x[i] >= 0.3*y[i])

In [545]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6260 rows, 6368 columns and 225881 nonzeros
Model fingerprint: 0x2b68d779
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1674 rows and 1434 columns
Presolve time: 0.58s
Presolved: 4586 rows, 4934 columns, 157100 nonzeros
Variable types: 2589 continuous, 2345 integer (2345 binary)

Root relaxation: objective 1.500000e+03, 88 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    6          - 1500.00000      -     -    0s
     0

In [546]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cod roe, smoked, semi-preserved") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Trout egges, canned") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tofu silky") <= 0)
for i in range(len(foods)):
    if tables[i][1] == "cakes and pastry":
        model.addConstr(x[i] >= 0.3*y[i])

In [547]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6348 rows, 6368 columns and 226054 nonzeros
Model fingerprint: 0x1bb8a863
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1596 rows and 1354 columns
Presolve time: 0.58s
Presolved: 4752 rows, 5014 columns, 157250 nonzeros
Variable types: 2586 continuous, 2428 integer (2428 binary)

Root relaxation: objective 1.500000e+03, 88 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    6          - 1500.00000      -     -    0s
H    0

In [548]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Black truffle, raw") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [549]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6349 rows, 6368 columns and 226055 nonzeros
Model fingerprint: 0x40038d91
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1599 rows and 1356 columns
Presolve time: 0.53s
Presolved: 4750 rows, 5012 columns, 157189 nonzeros
Variable types: 2585 continuous, 2427 integer (2427 binary)

Root relaxation: objective 1.500000e+03, 90 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    8          - 1500.00000      -     -    0s
H    0

In [550]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Crab, boiled/cooked in water") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [551]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6350 rows, 6368 columns and 226056 nonzeros
Model fingerprint: 0xaa408859
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1602 rows and 1358 columns
Presolve time: 0.77s
Presolved: 4748 rows, 5010 columns, 157128 nonzeros
Variable types: 2584 continuous, 2426 integer (2426 binary)

Root relaxation: objective 1.500000e+03, 90 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    8          - 1500.00000      -     -    1s
H    0

In [552]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Molokhia powder (dried jute leaves)") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [553]:
# Diet plan with nutrient and palatability constraints
# This diet plan is used as the basis regarding the diets for diabetics, people with cow's milk allergy and vegetarians
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6351 rows, 6368 columns and 226057 nonzeros
Model fingerprint: 0x7b259297
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve produced solution with objective 1500 (0.09s)
Loaded MIP start from previous solve with objective 1500

Presolve removed 1604 rows and 1360 columns
Presolve time: 0.62s
Presolved: 4747 rows, 5008 columns, 157066 nonzeros
Variable types: 2583 continuous, 2425 integer (2425 binary)

Root relaxation: cutoff, 18 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1

In [554]:
NutrientValues()

['Energy', 1500, 1499.9999999999998, 2874.5]
['Water', 3020, 3081.0695320206405, 6000]
['Protein', 62.8, 62.8, 250]
['Carbohydrate', 279.11, 279.11000000000007, 315]
['Fat', 33.33333333333333, 39.32853214190119, 58.33333333333332]
['FA Saturated', 0, 9.630381114314757, 16.666666666666664]
['FA Mono', 16.666666666666664, 16.666666666666664, 33.33333333333333]
['FA Poly', 4.999999999999999, 10.253692758815944, 19.999999999999996]
['Calcium', 930, 1904.7280027672841, 2500]
['Chloride', 2057.14, 3211.6928235629603, 3600]
['Copper', 1.11, 1.2250077960050778, 5]
['Iron', 9.87, 18.527535077784062, 45]
['Iodine', 154, 181.24926631692858, 600]
['Magnesium', 354, 416.1284320600978, 704]
['Manganese', 3.26, 4.221985057573389, 11]
['Phosphorus', 725, 1019.8195383211042, 4000]
['Potassium', 3580, 3690.7816323649, 1e+100]
['Selenium', 63.5, 118.70189920687884, 300]
['Sodium', 1702.22, 2270.243465166599, 2300]
['Zinc', 10.9, 12.30912841479495, 25]
['Vitamin A', 815, 814.9999999999998, 3000]
['Vitamin

In [555]:
NutrientPercentage()

7 4 12 3 10 5 4 0 0 2 0 3 32 3 4 2 0 2 Energy
4 4 4 3 0 1 4 68 4 4 0 0 0 0 0 0 0 0 Water
22 8 4 4 6 20 8 0 0 0 0 0 16 3 0 0 0 4 Protein
1 2 24 2 10 0 2 0 0 3 0 5 38 6 0 0 0 1 Carbohydrate
8 1 2 0 5 7 4 0 0 0 0 0 3 15 18 28 0 2 Fat
6 0 3 0 10 11 12 0 0 0 0 0 4 17 19 10 0 1 FA Saturated
10 0 0 0 2 6 2 0 0 0 0 0 4 12 20 39 0 0 FA Mono
9 2 1 1 1 2 0 0 0 0 0 0 7 19 17 31 0 5 FA Poly
1 2 0 2 0 0 9 22 0 0 0 38 19 1 0 0 0 0 Calcium
23 2 7 11 10 1 4 0 1 0 0 0 22 4 0 8 0 0 Chloride
20 5 15 6 5 4 0 2 0 2 0 1 22 5 0 0 0 6 Copper
4 6 7 6 2 3 0 0 0 0 0 5 52 4 0 0 0 4 Iron
58 0 3 0 1 1 8 0 0 0 0 1 5 1 16 1 0 0 Iodine
8 7 20 2 2 3 3 20 0 0 0 6 14 2 0 0 0 6 Magnesium
7 10 0 4 6 0 0 0 2 0 0 0 23 3 0 0 0 40 Manganese
14 9 7 4 4 10 12 0 0 0 0 1 19 3 0 0 0 9 Phosphorus
14 17 31 9 1 4 6 0 0 0 0 0 9 1 0 0 0 2 Potassium
4 1 3 1 0 4 12 54 0 0 0 4 7 2 0 1 0 0 Selenium
18 0 0 12 0 1 2 0 2 0 0 0 48 4 0 7 0 0 Sodium
15 4 2 2 22 14 4 1 0 0 0 0 17 2 0 0 0 11 Zinc
7 5 10 66 0 0 6 0 0 0 0 0 0 1 0 0 0 0 Vitamin A
1 0 0

In [374]:
# vegetarian diet
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "cooked meat") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "cooked ham") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "raw and cured ham") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "dry sausages") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "sausages") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "pates and terrines") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "rillettes") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "quenelles") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "other delicatessen") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "other meat products") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "fish, cooked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "seafood, cooked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "fish products") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "meat dishes, no garnish") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "meat dishes, with starchy food") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "fish dishes, no garnish") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "fish dishes, with starchy food") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [375]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6368 rows, 6368 columns and 226572 nonzeros
Model fingerprint: 0xa569ef62
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2590 rows and 2329 columns
Presolve time: 0.63s
Presolved: 3778 rows, 4039 columns, 126528 nonzeros
Variable types: 2098 continuous, 1941 integer (1941 binary)

Root relaxation: objective 1.500000e+03, 212 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    5          - 1500.00000      -     -    1s
H    

In [376]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "soft drinks") <= 5)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "fish oils") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Lard or pork fat") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Pork fat, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chicken fat") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Duck fat") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Goose fat") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Turkey fat") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [377]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6376 rows, 6368 columns and 226628 nonzeros
Model fingerprint: 0x73feb6de
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2607 rows and 2347 columns
Presolve time: 0.57s
Presolved: 3769 rows, 4021 columns, 124927 nonzeros
Variable types: 2089 continuous, 1932 integer (1932 binary)

Root relaxation: objective 1.500000e+03, 271 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1500.0000000 1500.00000  0.00%     -    0s

Expl

In [378]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chanterelle or girolle mushroom, raw") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [379]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6377 rows, 6368 columns and 226629 nonzeros
Model fingerprint: 0xa0340a79
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2610 rows and 2349 columns
Presolve time: 0.60s
Presolved: 3767 rows, 4019 columns, 124866 nonzeros
Variable types: 2088 continuous, 1931 integer (1931 binary)

Root relaxation: objective 1.500000e+03, 332 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    4          - 1500.00000      -     -    0s
H    

In [380]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Squash, butternut, peeled, cooked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tapenade (a puree of capers, pitted black olives, anchovy and herbs, with olive oil and lemon juice") <= 0)
for i in range(len(foods)):
    if tables[i][1] == "cereal bars":
        model.addConstr(x[i] >= 0.25*y[i])

In [381]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6387 rows, 6368 columns and 226647 nonzeros
Model fingerprint: 0xd8fe2ffc
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2606 rows and 2344 columns
Presolve time: 0.59s
Presolved: 3781 rows, 4024 columns, 124783 nonzeros
Variable types: 2086 continuous, 1938 integer (1938 binary)

Root relaxation: objective 1.500000e+03, 407 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    5          - 1500.00000      -     -    0s
H    

In [382]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Quince, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Sodium bicarbonate") <= 0.04)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cep or boletus mushroom, raw" or tables[i][3] == "Caesar's mushroom or royal agaric, raw") <= 2.50)

<gurobi.Constr *Awaiting Model Update*>

In [383]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6390 rows, 6368 columns and 226651 nonzeros
Model fingerprint: 0x8c4ae512
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2610 rows and 2346 columns
Presolve time: 0.54s
Presolved: 3780 rows, 4022 columns, 124725 nonzeros
Variable types: 2085 continuous, 1937 integer (1937 binary)

Root relaxation: objective 1.500000e+03, 353 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    9          - 1500.00000      -     -    0s
H    

In [384]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Broth, stock or bouillon, beef, dehydrated and reconstituted") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tarragon, fresh") <= 0.10)

<gurobi.Constr *Awaiting Model Update*>

In [385]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6392 rows, 6368 columns and 226653 nonzeros
Model fingerprint: 0x9912ddc1
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2614 rows and 2348 columns
Presolve time: 0.53s
Presolved: 3778 rows, 4020 columns, 124670 nonzeros
Variable types: 2084 continuous, 1936 integer (1936 binary)

Root relaxation: objective 1.500000e+03, 234 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    6          - 1500.00000      -     -    0s
H    

In [388]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Thyme, fresh") <= 0.05)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][2] == "eggs, raw") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [389]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6396 rows, 6368 columns and 226662 nonzeros
Model fingerprint: 0x94543f2a
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R6395 by 1.000000000

Presolve removed 2622 rows and 2352 columns
Presolve time: 0.55s
Presolved: 3774 rows, 4016 columns, 124551 nonzeros
Variable types: 2082 continuous, 1934 integer (1934 binary)

Root relaxation: objective 1.500000e+03, 296 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

In [390]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Parsley, fresh") <= 0.10)

<gurobi.Constr *Awaiting Model Update*>

In [391]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6397 rows, 6368 columns and 226663 nonzeros
Model fingerprint: 0x00b4463c
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve produced solution with objective 1576.26 (0.08s)
Loaded MIP start from previous solve with objective 1576.26

Presolve removed 2623 rows and 2352 columns
Presolve time: 0.53s
Presolved: 3774 rows, 4016 columns, 124551 nonzeros
Variable types: 2082 continuous, 1934 integer (1934 binary)

Root relaxation: objective 1.500000e+03, 321 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0

In [392]:
NutrientValues()

['Energy', 1500, 1500.0, 2874.5]
['Water', 3020, 3239.0295879713976, 6000]
['Protein', 62.8, 99.80359058235457, 250]
['Carbohydrate', 279.11, 280.0186809994316, 315]
['Fat', 33.333333333333336, 33.333333333333336, 58.333333333333336]
['FA Saturated', 0, 16.666666666666668, 16.666666666666668]
['FA Mono', 16.666666666666668, 28.977321945252505, 33.333333333333336]
['FA Poly', 5.0, 13.715954155597727, 20.0]
['Calcium', 930, 1901.9320370971832, 2500]
['Chloride', 2057.14, 2562.733042128791, 3600]
['Copper', 1.11, 3.2378249651754247, 5]
['Iron', 9.87, 32.269611553941346, 45]
['Iodine', 154, 154.0, 600]
['Magnesium', 354, 561.2691495458336, 704]
['Manganese', 3.26, 7.416308497339064, 11]
['Phosphorus', 725, 1992.8818780462934, 4000]
['Potassium', 3580, 5079.152876165842, 1e+100]
['Selenium', 63.5, 205.3501315489428, 300]
['Sodium', 1702.22, 1702.22, 2300]
['Zinc', 10.9, 12.957946852183426, 25]
['Vitamin A', 815, 882.9609988769412, 3000]
['Vitamin D', 11.75, 11.75, 100]
['Vitamin E', 11.9, 1

In [393]:
NutrientPercentage()

1 5 9 35 14 8 0 3 15 4 0 0 2 0 Energy
2 6 3 0 0 10 74 0 0 0 0 1 0 0 Water
2 7 10 52 4 12 0 0 4 3 0 0 0 0 Protein
0 2 7 34 16 6 0 5 17 8 0 0 0 0 Carbohydrate
1 3 0 11 5 0 0 0 1 35 0 22 17 0 Fat
0 1 0 3 1 0 0 0 1 23 57 5 4 0 FA Saturated
0 0 1 3 2 0 0 0 1 17 47 14 12 0 FA Mono
1 4 0 13 4 0 0 0 2 16 36 14 4 0 FA Poly
8 3 7 5 0 20 3 38 8 1 0 0 1 0 Calcium
3 6 4 5 3 14 0 0 13 1 0 17 18 10 Chloride
3 17 13 48 4 0 0 0 3 3 0 0 1 0 Copper
7 7 9 45 1 0 0 3 13 3 0 0 7 0 Iron
2 5 24 12 3 31 0 1 3 4 0 3 1 5 Iodine
8 5 13 32 16 7 1 4 4 3 0 0 0 0 Magnesium
7 5 13 32 21 0 8 0 6 2 0 0 0 0 Manganese
2 16 7 36 11 16 0 0 4 2 0 0 0 0 Phosphorus
7 17 21 30 3 11 0 0 3 2 0 0 0 0 Potassium
0 6 10 48 2 8 10 2 1 1 0 2 3 0 Selenium
2 0 3 0 2 8 0 0 29 3 0 18 19 10 Sodium
3 7 11 46 7 10 0 0 7 2 0 0 0 0 Zinc
66 0 26 0 0 0 0 0 0 3 0 0 0 0 Vitamin A
0 65 8 2 1 0 0 0 9 1 10 0 0 0 Vitamin D
16 0 10 5 3 0 0 0 27 11 3 10 9 0 Vitamin E
46 8 38 2 0 0 0 0 0 1 0 1 0 0 Vitamin K
18 6 38 0 0 2 0 0 28 0 0 3 0 0 Vitamin C
3 19 13

In [117]:
# To get this diet plan, first reset the model and create the palatability diet
# and skip the constraints for the vegetarian diet and go directly to this cell
# Allergic to cow's milk
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "milk and milk products") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "dairy beverages") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "butters") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "ice cream and sorbet") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Yogurt sauce") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tzatziki, from yogurt, prepacked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Yogurt cake") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Buttered and caramelized milk bread cake") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Potato puree, with milk and butter, unsalted") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Snail in parsley butter, prepacked, cooked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Croissant, butter, from bakery") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Butter biscuit (cookie)") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Butter biscuit (cookie), with chocolate") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Biscuit shortbread, with butter") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Biscuit shortbread, with butter and chocolate") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Butter pound cake, prepacked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Buttered and caramelized milk bread cake") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Madeleine cake, pure butter, prepacked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "White butter sauce, prepacked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Sauce, butter, prepacked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "margarines") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "chocolate and chocolate products") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Nougat") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "cakes and pastry") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "omelettes and other egg products") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [118]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6376 rows, 6368 columns and 226565 nonzeros
Model fingerprint: 0xe525dc05
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2512 rows and 2314 columns
Presolve time: 0.53s
Presolved: 3864 rows, 4054 columns, 125556 nonzeros
Variable types: 2086 continuous, 1968 integer (1968 binary)

Root relaxation: objective 1.500000e+03, 119 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    2          - 1500.00000      -     -    0s
H    

In [119]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Dandelion, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Quince, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chewing gum, without sugar") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chewing gum, sugar level unknown (average)") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Light syrup for canned pears") <= 0.20)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "breads") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [120]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6382 rows, 6368 columns and 226603 nonzeros
Model fingerprint: 0xeb24fe8c
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2588 rows and 2386 columns
Presolve time: 0.52s
Presolved: 3794 rows, 3982 columns, 123210 nonzeros
Variable types: 2049 continuous, 1933 integer (1933 binary)

Root relaxation: objective 1.500000e+03, 83 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    7          - 1500.00000      -     -    0s
H    0

In [121]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Artichoke base, frozen, raw") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [122]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6383 rows, 6368 columns and 226604 nonzeros
Model fingerprint: 0xf6e9503a
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2591 rows and 2388 columns
Presolve time: 0.54s
Presolved: 3792 rows, 3980 columns, 123145 nonzeros
Variable types: 2048 continuous, 1932 integer (1932 binary)

Root relaxation: objective 1.500000e+03, 69 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    7          - 1500.00000      -     -    0s
H    0

In [123]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cassava or manioc, roots, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Beef, oxtail, boiled/cooked in water") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "other meats") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [124]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6386 rows, 6368 columns and 226650 nonzeros
Model fingerprint: 0x9672f9dc
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2636 rows and 2430 columns
Presolve time: 0.54s
Presolved: 3750 rows, 3938 columns, 121869 nonzeros
Variable types: 2027 continuous, 1911 integer (1911 binary)

Root relaxation: objective 1.500000e+03, 120 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    5          - 1500.00000      -     -    0s
H    

In [125]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Taro, tuber, cooked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Royal jelly") <= 0.05)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Maize/corn bran") <= 0.15)

<gurobi.Constr *Awaiting Model Update*>

In [126]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6389 rows, 6368 columns and 226653 nonzeros
Model fingerprint: 0xb62cd388
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2642 rows and 2432 columns
Presolve time: 0.53s
Presolved: 3747 rows, 3936 columns, 119964 nonzeros
Variable types: 2026 continuous, 1910 integer (1910 binary)

Root relaxation: objective 1.500000e+03, 122 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    8          - 1500.00000      -     -    0s
     

In [127]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Spinach, raw" or tables[i][3] == "Spinach, boiled/cooked in water") <= 2.5)

<gurobi.Constr *Awaiting Model Update*>

In [128]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6390 rows, 6368 columns and 226655 nonzeros
Model fingerprint: 0x66fff455
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2642 rows and 2432 columns
Presolve time: 0.54s
Presolved: 3748 rows, 3936 columns, 119966 nonzeros
Variable types: 2026 continuous, 1910 integer (1910 binary)

Root relaxation: objective 1.500000e+03, 125 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    1          - 1500.00000      -     -    0s
H    

In [129]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cod liver, canned, drained") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "cereal bars") <= 0) # contains dairy

<gurobi.Constr *Awaiting Model Update*>

In [130]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6392 rows, 6368 columns and 226664 nonzeros
Model fingerprint: 0x2393a2d8
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2646 rows and 2442 columns
Presolve time: 0.52s
Presolved: 3746 rows, 3926 columns, 119451 nonzeros
Variable types: 2017 continuous, 1909 integer (1909 binary)

Root relaxation: objective 1.500000e+03, 125 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    6          - 1500.00000      -     -    0s
H    

In [133]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Flaked crab and/or decorticated crab leg, canned, drained") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Papaya, pulp, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Dulce de leche or confiture de lait") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [134]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6398 rows, 6368 columns and 226669 nonzeros
Model fingerprint: 0x8dfe5b3b
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2656 rows and 2447 columns
Presolve time: 0.55s
Presolved: 3742 rows, 3921 columns, 119275 nonzeros
Variable types: 2014 continuous, 1907 integer (1907 binary)

Root relaxation: objective 1.500000e+03, 125 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    5          - 1500.00000      -     -    0s
H    

In [135]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Gelling agent for jam") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Herbs, fresh (average)") <= 0.10)
for i in range(len(foods)):
    if tables[i][1] == "non-chocolate confectionery":
        model.addConstr(x[i] >= 0.1*y[i])

In [136]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6417 rows, 6368 columns and 226705 nonzeros
Model fingerprint: 0x67cf72ac
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2647 rows and 2434 columns
Presolve time: 0.53s
Presolved: 3770 rows, 3934 columns, 119274 nonzeros
Variable types: 2013 continuous, 1921 integer (1921 binary)

Root relaxation: objective 1.500000e+03, 125 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    7          - 1500.00000      -     -    0s
H    

In [137]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Salsify, frozen, raw") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [138]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6418 rows, 6368 columns and 226706 nonzeros
Model fingerprint: 0x5c76ff8a
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 2650 rows and 2436 columns
Presolve time: 0.71s
Presolved: 3768 rows, 3932 columns, 119209 nonzeros
Variable types: 2012 continuous, 1920 integer (1920 binary)

Root relaxation: objective 1.500000e+03, 125 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    4          - 1500.00000      -     -    1s
H    

In [139]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tarragon, fresh") <= 0.10)

<gurobi.Constr *Awaiting Model Update*>

In [140]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6419 rows, 6368 columns and 226707 nonzeros
Model fingerprint: 0x2ca80864
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve produced solution with objective 1500 (0.07s)
Loaded MIP start from previous solve with objective 1500

Presolve removed 2651 rows and 2436 columns
Presolve time: 0.55s
Presolved: 3768 rows, 3932 columns, 119209 nonzeros
Variable types: 2012 continuous, 1920 integer (1920 binary)

Root relaxation: cutoff, 32 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1

In [141]:
NutrientValues()

['Energy', 1500, 1499.9999999999998, 2874.5]
['Water', 3020, 3025.1255432957755, 6000]
['Protein', 62.8, 64.66647934272315, 250]
['Carbohydrate', 279.11, 279.11000000000007, 315]
['Fat', 33.33333333333333, 33.333333333333314, 58.33333333333332]
['FA Saturated', 0, 8.959837373604847, 16.666666666666664]
['FA Mono', 16.666666666666664, 16.666666666666657, 33.33333333333333]
['FA Poly', 4.999999999999999, 7.148321637782693, 19.999999999999996]
['Calcium', 930, 930.0000000000006, 2500]
['Chloride', 2057.14, 3185.781119394889, 3600]
['Copper', 1.11, 1.798557111869897, 5]
['Iron', 9.87, 34.98712013203292, 45]
['Iodine', 154, 600.0, 600]
['Magnesium', 354, 612.249427028464, 704]
['Manganese', 3.26, 6.052233723816071, 11]
['Phosphorus', 725, 1129.3990056776397, 4000]
['Potassium', 3580, 3974.3744374901476, 1e+100]
['Selenium', 63.5, 142.20605726204198, 300]
['Sodium', 1702.22, 2256.397055775055, 2300]
['Zinc', 10.9, 14.496082630376586, 25]
['Vitamin A', 815, 1107.468935244747, 3000]
['Vitamin 

In [142]:
NutrientPercentage()

3 3 13 17 10 5 0 4 0 2 23 8 1 3 0 0 0 0 0 Energy
4 7 5 0 0 1 79 0 0 0 0 0 1 0 0 0 0 0 0 Water
1 7 4 42 4 23 0 0 0 0 9 0 0 0 0 0 1 1 0 Protein
1 2 26 16 11 0 0 7 0 2 27 0 2 0 0 0 0 0 0 Carbohydrate
5 0 2 2 3 6 0 3 0 2 4 43 0 20 1 0 0 0 2 Fat
14 0 3 1 2 7 0 9 0 4 2 40 0 9 0 0 0 0 1 FA Saturated
2 0 0 1 2 4 0 0 0 2 2 40 7 31 1 0 0 0 0 FA Mono
0 1 2 5 6 2 0 0 0 2 6 48 5 8 1 1 0 1 5 FA Poly
2 14 0 3 0 0 49 1 0 1 0 0 1 2 1 9 8 1 0 Calcium
18 3 7 2 1 1 2 0 0 1 15 0 23 19 0 0 0 0 0 Chloride
2 12 11 45 5 1 1 0 0 1 2 0 2 2 2 2 3 1 0 Copper
1 4 4 18 1 0 0 0 0 0 20 0 1 3 1 7 33 0 0 Iron
1 4 1 1 0 0 0 0 0 0 0 9 0 0 0 0 78 0 0 Iodine
1 7 15 12 10 2 15 1 0 0 3 0 1 0 1 3 19 1 0 Magnesium
1 10 0 19 18 0 3 2 0 0 10 0 1 0 4 5 20 0 0 Manganese
2 8 7 29 14 9 0 0 0 1 11 0 1 0 0 0 4 4 2 Phosphorus
4 20 31 18 3 4 0 0 0 0 3 0 2 1 0 2 2 1 0 Potassium
7 17 3 7 2 4 47 0 0 0 4 0 1 0 0 0 0 0 0 Selenium
15 2 0 0 1 0 5 0 0 1 17 0 22 17 0 0 10 5 0 Sodium
1 4 2 26 4 6 1 0 0 0 43 0 0 0 0 1 2 2 0 Zinc
36 24 8 0 0 0 0 0 0

In [556]:
# To obtain a diet plan for people that have diabetes,
# follow the same steps as before: create palatability diet
# and then go this cell.
# upper bound of sugar is set to 25 g
# upper bound of cholesterol is set to 200 mg
model.addConstr(gp.quicksum(x[i]*tables[i][38] for i in range(len(tables))) <= 25) # sugar
model.addConstr(gp.quicksum(x[i]*tables[i][39] for i in range(len(tables))) <= 200) # cholesterol

<gurobi.Constr *Awaiting Model Update*>

In [557]:
energy = 2000 # a starter value is needed, otherwise we get an error
# The variable energy only appears in the function NutrientValues and in
# this function the current energy amount is taken to calculate the lower
# and upper bounds for fat and the fatty acids
inf = GRB.INFINITY # define infinity to variable 'inf', because some nutrients have no upper bound.
# Change lower bound of energy to 0, because otherwise the function NutrientValues
# would have a lower bound of 1500, with an actual value of 1154.09 which would be illogical
# Also, change the name of Upper_bound to Upper_bound_no_energy
Lower_bound_sugar_and_cholesterol = {'Energy': 0, 'Water': 3020, 'Protein': 62.8,
 'Carbohydrate': 279.11, 'Fat': energy*0.2/9, 'Saturated Fat': 0,
 'FA Mono': energy*0.1/9, 'FA Poly': energy*0.03/9, 'Calcium': 930,
 'Chloride': 2057.14, 'Copper': 1.11, 'Iron': 9.87, 'Iodine': 154,
 'Magnesium': 354, 'Manganese': 3.26, 'Phosphorus': 725, 'Potassium': 3580,
 'Selenium': 63.5, 'Sodium': 1702.22, 'Zinc': 10.9, 'Vitamin A': 815,
 'Vitamin D': 11.75, 'Vitamin E': 11.9, 'Vitamin K': 92.9, 'Vitamin C': 82,
 'Vitamin B1': 1.2, 'Vitamin B2': 1.415, 'Vitamin B3': 16.45, 
 'Vitamin B5': 5.31, 'Vitamin B6': 1.51, 'Vitamin B9': 315, 'Vitamin B12': 2.64,
 'Fibres': 30.16, 'Salt': 0, 'Sugar': 0, 'Cholesterol': 0}
Upper_bound_sugar_and_cholesterol = {'Energy': 2874.5, 'Water': 6000, 'Protein': 250,
 'Carbohydrate': 315, 'Fat': energy*0.35/9, 'Saturated Fat': energy*0.1/9,
 'FA Mono': energy*0.2/9, 'FA Poly': energy*0.12/9, 'Calcium': 2500,
 'Chloride': 3600, 'Copper': 5, 'Iron': 45, 'Iodine': 600, 'Magnesium': 704,
 'Manganese': 11, 'Phosphorus': 4000, 'Potassium': inf, 'Selenium': 300,
 'Sodium': 2300, 'Zinc': 25, 'Vitamin A': 3000, 'Vitamin D': 100,
 'Vitamin E': 300, 'Vitamin K': inf, 'Vitamin C': 2000, 'Vitamin B1': inf,
 'Vitamin B2': inf, 'Vitamin B3': 900, 'Vitamin B5': 200, 'Vitamin B6': 25,
 'Vitamin B9': 1000, 'Vitamin B12': inf, 'Fibres': inf, 'Salt': 6,
 'Sugar': 25, 'Cholesterol': 200}

In [558]:
# Create function that calculates amount of each nutrient based on diet plan
# This function is altered to change the lower bound of energy to 0
def NutrientValues3():
    nutrient_value_list = [0]*36 # create empty list
    if model.status == GRB.OPTIMAL: # if model is feasible
        for i in range(len(foods)): # loop over every food
            if x[i].x > 0.00001: # if food is in diet plan
                for j in range(len(nutrients)): # then loop over nutrients
                    # multiply every nutrient value by amount in diet and add it to the list
                    nutrient_value_list[j] += df.iloc[i,j+4]*x[i].x
    
    energy = nutrient_value_list[0] # Energy amount in diet plan
    # calculate lower bounds for fat and FA mono/poly
    Lower_bound_no_energy["Fat"] = energy*0.2/9
    Lower_bound_no_energy["FA Mono"] = energy*0.1/9
    Lower_bound_no_energy["FA Poly"] = energy*0.03/9
    lower_bound_list = Lower_bound_sugar_and_cholesterol.values() # create list of lower bound values

    # calculate upper bounds for fat, saturated fat and FA mono/poly
    Upper_bound_no_energy["Fat"] = energy*0.35/9
    Upper_bound_no_energy["Saturated Fat"] = energy*0.1/9
    Upper_bound_no_energy["FA Mono"] = energy*0.2/9
    Upper_bound_no_energy["FA Poly"] = energy*0.12/9
    upper_bound_list = Upper_bound_sugar_and_cholesterol.values() # create list of upper bound values
    # w = nutrient, x = lb, y = amount in diet plan, z = upper bound
    nutrient_amount = [[w,x,y,z] for w, x, y, z in zip(nutrients, lower_bound_list, nutrient_value_list, upper_bound_list)]
    print(*nutrient_amount, sep = "\n") # every list is printed on a new line


In [559]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6353 rows, 6368 columns and 230960 nonzeros
Model fingerprint: 0xe89938a6
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1654 rows and 1408 columns
Presolve time: 0.63s
Presolved: 4699 rows, 4960 columns, 155579 nonzeros
Variable types: 2559 continuous, 2401 integer (2401 binary)

Root relaxation: objective 1.500000e+03, 131 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    2          - 1500.00000      -     -    0s
H    

In [560]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Quince, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Taro, tuber, cooked") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [561]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6355 rows, 6368 columns and 230962 nonzeros
Model fingerprint: 0xa8196e9e
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1660 rows and 1412 columns
Presolve time: 0.61s
Presolved: 4695 rows, 4956 columns, 155457 nonzeros
Variable types: 2557 continuous, 2399 integer (2399 binary)

Root relaxation: objective 1.500000e+03, 97 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    6          - 1500.00000      -     -    0s
H    0

In [562]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Artichoke base, frozen, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cassava or manioc, roots, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Ox muzzle") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [563]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6358 rows, 6368 columns and 230965 nonzeros
Model fingerprint: 0xa04d1240
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1669 rows and 1418 columns
Presolve time: 0.56s
Presolved: 4689 rows, 4950 columns, 155267 nonzeros
Variable types: 2554 continuous, 2396 integer (2396 binary)

Root relaxation: objective 1.500000e+03, 126 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    5          - 1500.00000      -     -    0s
H    

In [564]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Royal jelly") <= 0.05)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Spiny lobster, boiled/cooked in water") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [565]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6360 rows, 6368 columns and 230967 nonzeros
Model fingerprint: 0x0ba98ee7
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1674 rows and 1420 columns
Presolve time: 0.58s
Presolved: 4686 rows, 4948 columns, 152833 nonzeros
Variable types: 2553 continuous, 2395 integer (2395 binary)

Root relaxation: objective 1.500000e+03, 468 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0    7          - 1500.00000      -     -    0s
H    

In [566]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Squash, butternut, pulp, raw") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [567]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6361 rows, 6368 columns and 230968 nonzeros
Model fingerprint: 0x9cb4969d
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1677 rows and 1422 columns
Presolve time: 0.62s
Presolved: 4684 rows, 4946 columns, 152772 nonzeros
Variable types: 2552 continuous, 2394 integer (2394 binary)

Root relaxation: objective 1.500000e+03, 565 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1500.00000    0   10          - 1500.00000      -     -    1s
H    

In [568]:
NutrientValues3()

['Energy', 0, 1499.9999999999998, 2874.5]
['Water', 3020, 3071.785569226503, 6000]
['Protein', 62.8, 79.48731397620452, 250]
['Carbohydrate', 279.11, 279.11, 315]
['Fat', 44.44444444444444, 34.634497153522084, 77.77777777777777]
['FA Saturated', 0, 10.523602602319158, 22.22222222222222]
['FA Mono', 22.22222222222222, 16.666666666666668, 44.44444444444444]
['FA Poly', 6.666666666666667, 9.88384702834746, 26.666666666666668]
['Calcium', 930, 1597.8056540425275, 2500]
['Chloride', 2057.14, 2057.14, 3600]
['Copper', 1.11, 1.9337706638892755, 5]
['Iron', 9.87, 23.371152798957723, 45]
['Iodine', 154, 241.00271095462435, 600]
['Magnesium', 354, 620.7153406888344, 704]
['Manganese', 3.26, 9.126811615882882, 11]
['Phosphorus', 725, 1375.6035757164632, 4000]
['Potassium', 3580, 5576.532236883398, 1e+100]
['Selenium', 63.5, 187.2190716277297, 300]
['Sodium', 1702.22, 1702.22, 2300]
['Zinc', 10.9, 11.090511374334994, 25]
['Vitamin A', 815, 1348.2186688910426, 3000]
['Vitamin D', 11.75, 11.75000000

In [569]:
NutrientPercentage()

0 3 12 17 13 20 9 6 0 4 0 3 3 0 2 0 0 0 Energy
3 7 5 0 3 0 1 1 45 31 0 0 0 0 0 0 0 0 Water
0 8 3 34 5 7 24 8 0 3 0 0 2 0 0 0 0 0 Protein
0 2 25 16 15 22 0 0 0 4 0 5 6 0 0 0 0 0 Carbohydrate
1 3 2 2 2 6 22 23 0 0 0 0 17 0 15 0 0 0 Fat
1 1 3 1 4 4 20 20 0 0 0 0 15 19 5 0 0 0 FA Saturated
0 0 0 1 2 5 13 19 0 0 0 0 12 18 24 0 0 0 FA Mono
0 6 1 3 1 9 27 12 0 0 0 0 19 11 4 0 0 0 FA Poly
0 17 0 1 0 0 6 1 18 2 0 46 1 0 0 0 0 1 Calcium
25 11 11 3 0 6 6 0 0 2 0 0 7 0 23 0 0 0 Chloride
0 12 10 42 7 9 5 1 0 2 0 0 3 0 1 0 0 1 Copper
0 38 6 26 0 3 5 4 0 1 0 4 3 0 4 0 0 0 Iron
14 3 2 4 5 3 10 12 0 41 0 0 1 0 0 0 0 0 Iodine
0 21 14 11 0 20 4 0 2 14 0 4 1 0 0 0 0 1 Magnesium
0 35 0 13 3 24 0 0 12 7 0 0 1 0 0 0 0 0 Manganese
0 8 6 23 2 23 17 4 0 5 0 0 2 0 0 0 0 3 Phosphorus
0 22 21 13 0 4 5 1 0 26 0 0 1 0 1 0 0 0 Potassium
0 0 2 5 7 4 15 2 3 53 0 2 1 0 0 0 0 0 Selenium
25 9 0 0 0 3 6 4 0 1 0 0 5 0 17 0 17 6 Sodium
0 18 3 35 7 12 9 3 0 2 0 0 2 0 0 0 0 3 Zinc
0 86 6 0 0 0 0 3 0 0 0 0 1 0 0 0 0 0 Vitamin A

In [462]:
# Set energy amount equal to 1500 to make the optimization go faster and minimize amount of food x[i] in the diet
# Set quantity of beverages to be 2400, to prevent fluctuation of the objective value
# First create the palatability diet and then start running from this cell
model.setObjective(gp.quicksum(x[i] for i in range(len(tables))), GRB.MINIMIZE)
model.addConstr(gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))) == 1500)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "beverages") == 24)

<gurobi.Constr *Awaiting Model Update*>

In [463]:
# Create function that prints objective value and the diet plan
# Change the function so that it prints the amount of food in a diet
def DietPlanSolutionAmount():
    if model.status == GRB.OPTIMAL: # if model has a solution/is feasible
        # print objective value
        print('\nObjective Value:')
        # objective function can change when creating a new model
        # for example, create a diet that minimizes amount of foods
        FoodAmount = model.ObjVal*100 - 2400 # calculate amount of food in the diet plan
        print('\nAmount of Food: %g' % FoodAmount)
        print('\nDiet Plan:')
        for i in range(len(foods)): # loop over every food
            if x[i].x > 0.00001: # if food is in diet plan
                # print food name and amount in the diet plan and multiply by 100
                # because ciqual tables provide every nutrient value per 100g
                print(tables[i][3], x[i].x*100)

In [464]:
model.optimize()
DietPlanSolutionAmount()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6353 rows, 6368 columns and 229439 nonzeros
Model fingerprint: 0x388e45a7
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve produced solution with objective 34.4228 (0.09s)
Loaded MIP start from previous solve with objective 34.4228

Presolve removed 1610 rows and 1362 columns
Presolve time: 0.76s
Presolved: 4743 rows, 5006 columns, 152150 nonzeros
Variable types: 2582 continuous, 2424 integer (2424 binary)

Root relaxation: objective 3.327343e+01, 237 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0

In [465]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Artichoke base, frozen, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Flaked crab and/or decorticated crab leg, canned, drained") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [466]:
model.optimize()
DietPlanSolutionAmount()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6355 rows, 6368 columns and 229441 nonzeros
Model fingerprint: 0x6ca89509
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1616 rows and 1366 columns
Presolve time: 0.62s
Presolved: 4739 rows, 5002 columns, 152032 nonzeros
Variable types: 2580 continuous, 2422 integer (2422 binary)

Root relaxation: objective 3.327546e+01, 243 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   33.27546    0   13          -   33.27546      -     -    0s
     

In [467]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Royal jelly") <= 0.05)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Spiny lobster, boiled/cooked in water") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Salsify, cooked") <= 0)
for i in range(len(foods)):
    if tables[i][1] == "ice cream":
        model.addConstr(x[i] >= 0.5*y[i])

In [468]:
model.optimize()
DietPlanSolutionAmount()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6370 rows, 6368 columns and 229468 nonzeros
Model fingerprint: 0x1382416c
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1611 rows and 1358 columns
Presolve time: 0.75s
Presolved: 4759 rows, 5010 columns, 151960 nonzeros
Variable types: 2578 continuous, 2432 integer (2432 binary)

Root relaxation: objective 3.329083e+01, 258 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   33.29083    0   12          -   33.29083      -     -    1s
     

In [469]:
for i in range(len(foods)):
    if tables[i][3] == "Chewing gum, without sugar":
        model.addConstr(x[i] >= 0.014*y[i])

In [470]:
model.optimize()
DietPlanSolutionAmount()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6371 rows, 6368 columns and 229470 nonzeros
Model fingerprint: 0x96eacfd7
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve produced solution with objective 33.3485 (0.10s)
Loaded MIP start from previous solve with objective 33.3485

Presolve removed 1610 rows and 1357 columns
Presolve time: 0.74s
Presolved: 4761 rows, 5011 columns, 151964 nonzeros
Variable types: 2578 continuous, 2433 integer (2433 binary)

Root relaxation: objective 3.329083e+01, 259 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0

In [471]:
NutrientValues()

['Energy', 1500, 1500.0000000000002, 2874.5]
['Water', 3020, 3020.0, 6000]
['Protein', 62.8, 62.79999999999998, 250]
['Carbohydrate', 279.11, 279.11, 315]
['Fat', 33.33333333333334, 33.33333333333334, 58.333333333333336]
['FA Saturated', 0, 9.102562735614194, 16.66666666666667]
['FA Mono', 16.66666666666667, 19.41894716102515, 33.33333333333334]
['FA Poly', 5.000000000000001, 12.604670407010536, 20.000000000000004]
['Calcium', 930, 929.9999999999999, 2500]
['Chloride', 2057.14, 3118.58494337539, 3600]
['Copper', 1.11, 1.5206796453597762, 5]
['Iron', 9.87, 15.588962388311005, 45]
['Iodine', 154, 154.0, 600]
['Magnesium', 354, 373.5509803193446, 704]
['Manganese', 3.26, 5.930491903717514, 11]
['Phosphorus', 725, 962.6243008858722, 4000]
['Potassium', 3580, 3580.0, 1e+100]
['Selenium', 63.5, 192.925414992048, 300]
['Sodium', 1702.22, 2279.5196685985848, 2300]
['Zinc', 10.9, 11.803996313340374, 25]
['Vitamin A', 815, 845.5187348141865, 3000]
['Vitamin D', 11.75, 11.749999999999998, 100]
['

In [472]:
NutrientPercentage()

45 2 11 5 13 5 2 0 0 0 0 4 0 6 0 0 0 0 0 0 0 Energy
3 4 4 2 2 1 1 68 5 5 0 0 0 0 0 0 0 0 0 0 0 Water
35 5 4 1 11 24 11 0 0 0 0 0 0 2 0 0 0 0 0 0 1 Protein
48 1 22 9 13 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 Carbohydrate
9 2 2 1 26 6 1 0 0 0 0 24 6 17 0 0 0 0 0 0 0 Fat
15 0 3 0 4 6 1 0 0 0 0 22 3 42 0 0 0 0 0 0 0 FA Saturated
49 0 0 0 6 4 1 0 0 0 0 19 6 10 0 0 0 0 0 0 0 FA Mono
47 2 1 0 10 0 0 0 0 0 0 15 4 13 0 0 0 0 0 0 0 FA Poly
20 16 0 2 11 0 4 35 0 0 0 0 0 1 0 0 0 0 0 0 1 Calcium
43 4 6 1 8 1 15 1 0 1 0 0 4 9 0 0 0 0 0 0 0 Chloride
34 8 11 4 22 6 2 1 0 1 0 0 0 2 0 0 0 0 0 0 1 Copper
30 31 8 4 5 12 2 0 0 0 0 0 0 1 0 0 0 1 0 0 0 Iron
21 3 3 0 14 2 17 0 10 0 0 21 0 2 0 0 0 0 0 0 0 Iodine
29 19 21 3 5 3 2 2 3 1 0 0 0 1 0 0 0 0 0 0 3 Magnesium
28 29 0 5 23 0 0 0 1 6 0 0 0 0 0 1 0 0 0 0 0 Manganese
37 6 7 2 14 10 10 0 1 0 0 0 0 2 0 0 0 0 0 1 4 Phosphorus
19 19 29 5 5 4 1 0 6 1 0 0 0 2 0 0 0 0 0 0 1 Potassium
10 0 2 0 8 0 11 53 8 1 0 0 0 1 0 0 0 0 0 0 0 Selenium
52 3 0 0 19 1 1 0 0 0 0 0 4 8 0 0 

In [280]:
# Set energy amount equal to 1500 and minimize numbers of food y[i] in the diet
# Again create palatability diet and then run from this cell
model.setObjective(gp.quicksum(y[i] for i in range(len(tables))), GRB.MINIMIZE)
model.addConstr(gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))) == 1500)

<gurobi.Constr *Awaiting Model Update*>

In [281]:
# Create function that prints objective value and the diet plan
# Function is changed to print the number of foods in the diet
def DietPlanSolutionNumber():
    if model.status == GRB.OPTIMAL: # if model has a solution/is feasible
        # print objective value
        print('\nObjective Value:')
        print('\nNumber of Foods: %g' % model.ObjVal)
        print('\nDiet Plan:')
        for i in range(len(foods)): # loop over every food
            if x[i].x > 0.00001: # if food is in diet plan
                # print food name and amount in the diet plan and multiply by 100
                # because ciqual tables provide every nutrient value per 100g
                print(tables[i][3], x[i].x*100)

In [282]:
model.optimize()
DietPlanSolutionNumber()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6352 rows, 6368 columns and 229144 nonzeros
Model fingerprint: 0x48184911
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve produced solution with objective 280 (0.10s)
Loaded MIP start from previous solve with objective 280

Presolve removed 1451 rows and 1204 columns
Presolve time: 0.76s
Presolved: 4901 rows, 5164 columns, 152470 nonzeros
Found heuristic solution: objective 18.0000000
Variable types: 2582 continuous, 2582 integer (2582 binary)

Root relaxation: objective 3.500718e+00, 2491 iterations, 0.46 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [283]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Mare's milk whole") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Artichoke base, frozen, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cod liver, canned, drained") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [284]:
model.optimize()
DietPlanSolutionNumber()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6355 rows, 6368 columns and 229146 nonzeros
Model fingerprint: 0x0026762d
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1458 rows and 1208 columns
Presolve time: 0.76s
Presolved: 4897 rows, 5160 columns, 152344 nonzeros
Variable types: 2580 continuous, 2580 integer (2580 binary)

Root relaxation: objective 3.589483e+00, 2253 iterations, 0.36 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.58948    0   19          -    3.58948      -     -    1s
H   

In [285]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Clam, boiled/cooked in water") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chanterelle or girolle mushroom, raw") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [286]:
model.optimize()
DietPlanSolutionNumber()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6357 rows, 6368 columns and 229148 nonzeros
Model fingerprint: 0xd27453ed
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1464 rows and 1212 columns
Presolve time: 0.61s
Presolved: 4893 rows, 5156 columns, 152224 nonzeros
Variable types: 2578 continuous, 2578 integer (2578 binary)

Root relaxation: objective 3.590074e+00, 2623 iterations, 0.32 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.59007    0   19          -    3.59007      -     -    1s
H   

In [287]:
NutrientValues()

['Energy', 1500, 1500.0, 2874.5]
['Water', 3020, 3020.0, 6000]
['Protein', 62.8, 64.03833529041303, 250]
['Carbohydrate', 279.11, 279.11000000000007, 315]
['Fat', 33.333333333333336, 53.48351966066118, 58.333333333333336]
['FA Saturated', 0, 14.780021236292118, 16.666666666666668]
['FA Mono', 16.666666666666668, 19.73172290440072, 33.333333333333336]
['FA Poly', 5.0, 12.610050776298456, 20.0]
['Calcium', 930, 1269.5741501631803, 2500]
['Chloride', 2057.14, 2445.494420376442, 3600]
['Copper', 1.11, 2.1290284455196185, 5]
['Iron', 9.87, 31.431393191796033, 45]
['Iodine', 154, 202.52317500650017, 600]
['Magnesium', 354, 669.6529696372593, 704]
['Manganese', 3.26, 10.463214022896052, 11]
['Phosphorus', 725, 1742.529715705414, 4000]
['Potassium', 3580, 4497.2379141242245, 1e+100]
['Selenium', 63.5, 235.39824456083184, 300]
['Sodium', 1702.22, 1966.6930120867926, 2300]
['Zinc', 10.9, 12.479457109654271, 25]
['Vitamin A', 815, 1549.8821040983746, 3000]
['Vitamin D', 11.75, 17.847569497845072,

In [289]:
NutrientPercentage()

10 3 16 47 11 0 10 Energy
5 6 5 0 2 79 0 Water
8 8 28 21 21 0 12 Protein
8 1 12 53 1 0 22 Carbohydrate
6 1 0 10 21 0 59 Fat
5 0 0 6 16 0 70 FA Saturated
5 0 2 10 11 0 69 FA Mono
2 4 0 17 28 0 48 FA Poly
3 18 18 1 20 30 6 Calcium
43 8 8 11 22 2 3 Chloride
10 9 34 19 7 1 15 Copper
38 23 15 6 8 0 8 Iron
14 3 30 9 33 0 9 Iodine
7 15 18 43 4 1 8 Magnesium
3 25 16 49 1 0 4 Manganese
11 5 14 43 17 0 8 Phosphorus
10 22 41 12 7 0 6 Potassium
8 0 15 7 13 50 3 Selenium
50 6 4 6 21 1 8 Sodium
19 13 20 26 11 2 7 Zinc
6 61 25 1 0 0 5 Vitamin A
2 0 9 2 83 0 2 Vitamin D
10 24 15 9 13 0 26 Vitamin E
1 65 31 0 0 0 1 Vitamin K
28 37 32 0 0 0 0 Vitamin C
27 11 40 10 0 0 9 Vitamin B1
14 25 20 2 21 0 16 Vitamin B2
17 7 20 12 34 0 5 Vitamin B3
9 6 31 24 16 0 10 Vitamin B5
11 22 27 17 15 0 6 Vitamin B6
5 51 27 9 0 0 6 Vitamin B9
11 0 0 0 85 0 2 Vitamin B12
19 12 43 14 0 0 9 Fibres
50 6 4 6 21 1 8 Salt
5 1 0 2 2 0 87 Sugar
19 0 0 1 20 0 58 Cholesterol


In [300]:
# Maximize satiety
# Create only diet with nutrient constraints
# palatability constraints will be added after the first diet
model.setObjective(gp.quicksum(x[i]*(100-tables[i][5]) for i in range(len(tables))), GRB.MAXIMIZE)
model.addConstr(gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))) == 1500)

<gurobi.Constr *Awaiting Model Update*>

In [301]:
# Create function that prints objective value and the diet plan
def DietPlanSolutionSatiety():
    if model.status == GRB.OPTIMAL: # if model has a solution/is feasible
        # print objective value
        print('\nObjective Value:')
        # objective function can change when creating a new model
        # for example, create a diet that minimizes amount of foods
        print('\nAmountDryFood: %g' % model.ObjVal)
        print('\nDiet Plan:')
        for i in range(len(foods)): # loop over every food
            if x[i].x > 0.00001: # if food is in diet plan
                # print food name and amount in the diet plan and multiply by 100
                # because ciqual tables provide every nutrient value per 100g
                print(tables[i][3], x[i].x*100)

In [302]:
model.optimize()
DietPlanSolutionSatiety()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3256 rows, 6368 columns and 217059 nonzeros
Model fingerprint: 0x0ec8a5a8
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [2e-02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]

MIP start from previous solve produced solution with objective 819.576 (0.52s)
Loaded MIP start from previous solve with objective 819.576

Presolve removed 3197 rows and 3191 columns
Presolve time: 0.48s
Presolved: 59 rows, 3177 columns, 173873 nonzeros
Variable types: 3177 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 70 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       81

In [303]:
# Add the first 2 batches of constraints together to save some time adding palatability constraints
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "seafood, raw") <= 0) # Create a constraint which excludes a food subgroup from the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "beverages") <= 24) # Create a constraint that puts an upper bound on a food group
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tapioca, raw") <= 0) # Constraint that excludes a specific food from the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "offals") <= 0) # Create a constraint which excludes a food subsubgroup from the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "coffee, tea, cocoa beverages, etc. ready to drink") <= 10)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Baking powder or raising agent") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "fish, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Baker's yeast, dehydrated") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Baker's yeast, compressed") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "meat, egg and fish") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "fats and oils") <= 1) # Create an upper bound for the number of fats and oils in the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "sauces") <= 0.5)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "vegetables from Martinique and their products") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "fruits from Martinique and their products") <= 0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "meat, egg and fish") <= 4)
for i in range(len(foods)):
    if tables[i][0] == "meat, egg and fish": # If a food from the food group 'meat, egg and fish' is present in the diet plan,
        model.addConstr(x[i] >= 0.5*y[i]) # Then the amount of the food has to be at least 50 g to avoid having small portions
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "seaweed") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "seaweed":
        model.addConstr(x[i] >= 0.05*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "vegetables") <= 5.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "vegetables") <= 5)
for i in range(len(foods)):
    if tables[i][1] == "vegetables":
        model.addConstr(x[i] >= 1.0*y[i])
for i in range(len(foods)):
    if tables[i][1] == "vegetables":
        model.addConstr(x[i] <= 2.5*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "fruits") <= 3.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "fruits") <= 3)
for i in range(len(foods)):
    if tables[i][1] == "fruits":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "nuts and seeds") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "nuts and seeds":
        model.addConstr(x[i] >= 0.30*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "baby food") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "potatoes and other tubers") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "potatoes and other tubers") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "potatoes and other tubers":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "legumes") <= 2.0)        
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "legumes") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "legumes":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "breakfast cereals") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "breakfast cereals":
        model.addConstr(x[i] >= 0.5*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "cooking aids") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "condiments") <= 0.30)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "foods for particular nutritional uses") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "cooking ingredients") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "alcoholic beverages") <= 5)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "beverages") <= 3)
for i in range(len(foods)):
    if tables[i][0] == "beverages":
        model.addConstr(x[i] >= 1.5*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "sweet biscuits") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "sweet biscuits":
        model.addConstr(x[i] >= 0.1*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "pasta, rice and grains") <= 1.50)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "pasta, rice and grains, raw") <= 0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "pasta, rice and grains") <= 2)
for i in range(len(foods)):
    if tables[i][1] == "pasta, rice and grains":
        model.addConstr(x[i] >= 0.75*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "savoury biscuits") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "savoury biscuits":
        model.addConstr(x[i] >= 0.50*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "starters and dishes") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "starters and dishes") <= 2)
for i in range(len(foods)):
    if tables[i][0] == "starters and dishes":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "milk") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "milk":
        model.addConstr(x[i] >= 1.5*y[i])        
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "dairy products and similar") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "dairy products and similar") <= 3)
for i in range(len(foods)):
    if tables[i][1] == "dairy products and similar":
        model.addConstr(x[i] >= 0.50*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "fromages blanc" or tables[i][1] == "cheese and similar") <= 0.60)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "cheese and similar") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "cheese and similar":
        model.addConstr(x[i] >= 0.30*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "herbs" or tables[i][1] == "spices") <= 0.3)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "herbs" or tables[i][1] == "spices") <= 2)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "raw meat") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "fats and oils") <= 0.4)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "sauces") <= 2)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "breads and similar") <= 2)
for i in range(len(foods)):
    if tables[i][1] == "breads and similar":
        model.addConstr(x[i] >= 0.4*y[i])

In [304]:
model.optimize()
DietPlanSolutionSatiety()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6211 rows, 6368 columns and 228883 nonzeros
Model fingerprint: 0x467c8591
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [2e-02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 6e+03]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R3266 by 74.000000000

Presolve removed 1680 rows and 1451 columns
Presolve time: 0.63s
Presolved: 4531 rows, 4917 columns, 152550 nonzeros
Variable types: 2598 continuous, 2319 integer (2319 binary)

Root relaxation: objective 7.464604e+02, 1254 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

In [305]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Morel, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Artichoke base, frozen, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chewing gum, without sugar") <= 0.20)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Maize/corn bran") <= 0.15)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Sodium bicarbonate") <= 0.04)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tarragon, fresh") <= 0.10)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Aspartame sweetener, in tablets" or tables[i][3] == "Stevia sweetener" 
                            or tables[i][3] == "Aspartame, in powder" or tables[i][3] == "Saccharin sweetener") <= 0.0032)

<gurobi.Constr *Awaiting Model Update*>

In [306]:
model.optimize()
DietPlanSolutionSatiety()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6218 rows, 6368 columns and 228893 nonzeros
Model fingerprint: 0x72b4e669
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [2e-02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1690 rows and 1455 columns
Presolve time: 0.59s
Presolved: 4528 rows, 4913 columns, 152430 nonzeros
Variable types: 2596 continuous, 2317 integer (2317 binary)

Root relaxation: objective 6.675612e+02, 2055 iterations, 0.13 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  667.56117    0    5          -  667.56117      -     -    0s
H   

In [307]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Quince, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cod roe, smoked, semi-preserved") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [308]:
model.optimize()
DietPlanSolutionSatiety()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6220 rows, 6368 columns and 228895 nonzeros
Model fingerprint: 0xc9e79f23
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [2e-02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1696 rows and 1459 columns
Presolve time: 0.62s
Presolved: 4524 rows, 4909 columns, 152309 nonzeros
Variable types: 2594 continuous, 2315 integer (2315 binary)

Root relaxation: objective 6.662810e+02, 1841 iterations, 0.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  666.28096    0    7          -  666.28096      -     -    0s
    

In [309]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chanterelle or girolle mushroom, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tomato sauce for pizza base") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Garlic, fresh") <= 0.04)

<gurobi.Constr *Awaiting Model Update*>

In [310]:
model.optimize()
DietPlanSolutionSatiety()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6223 rows, 6368 columns and 228898 nonzeros
Model fingerprint: 0xa740dc91
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [2e-02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1701 rows and 1462 columns
Presolve time: 0.56s
Presolved: 4522 rows, 4906 columns, 152191 nonzeros
Variable types: 2592 continuous, 2314 integer (2314 binary)

Root relaxation: objective 6.662810e+02, 1840 iterations, 0.12 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  666.28096    0    7          -  666.28096      -     -    0s
H   

In [311]:
NutrientValues()

['Energy', 1500, 1500.0, 2874.5]
['Water', 3020, 3019.9999999999995, 6000]
['Protein', 62.8, 101.74666159693174, 250]
['Carbohydrate', 279.11, 279.11, 315]
['Fat', 33.333333333333336, 33.33333333333333, 58.333333333333336]
['FA Saturated', 0, 16.666666666666657, 16.666666666666668]
['FA Mono', 16.666666666666668, 31.571731503696842, 33.333333333333336]
['FA Poly', 5.0, 16.890538479894165, 20.0]
['Calcium', 930, 1400.5399933167503, 2500]
['Chloride', 2057.14, 2462.366081275194, 3600]
['Copper', 1.11, 3.852798904489651, 5]
['Iron', 9.87, 31.10141531110906, 45]
['Iodine', 154, 178.30485967769764, 600]
['Magnesium', 354, 703.9999999999999, 704]
['Manganese', 3.26, 10.999999999999998, 11]
['Phosphorus', 725, 2041.410815544575, 4000]
['Potassium', 3580, 7554.6184058389335, 1e+100]
['Selenium', 63.5, 191.32576223555245, 300]
['Sodium', 1702.22, 2300.0000000000005, 2300]
['Zinc', 10.9, 15.617910085478618, 25]
['Vitamin A', 815, 954.4939846950542, 3000]
['Vitamin D', 11.75, 12.573277983188921, 

In [312]:
NutrientPercentage()

1 1 13 8 9 33 7 3 4 0 4 0 0 3 2 0 1 0 0 0 0 2 0 Energy
3 3 5 3 5 0 3 1 1 31 8 31 0 0 0 0 0 0 0 0 0 0 0 Water
0 3 3 9 3 49 3 10 10 0 1 0 0 0 1 0 0 0 0 0 0 1 0 Protein
1 0 26 6 10 31 6 0 0 0 2 0 0 5 5 0 0 0 0 0 0 0 0 Carbohydrate
0 1 2 0 1 10 6 2 9 0 2 0 0 0 23 0 20 16 0 0 0 0 0 Fat
0 0 2 0 1 2 2 2 4 0 0 4 0 0 15 53 3 4 0 0 0 0 0 FA Saturated
0 0 0 0 3 2 2 1 3 0 0 6 0 0 10 40 13 9 2 0 0 0 0 FA Mono
0 0 1 0 2 11 4 1 5 0 0 12 0 0 8 27 11 8 0 0 0 0 0 FA Poly
0 0 0 9 2 9 0 0 2 0 2 7 0 52 1 0 0 0 0 0 6 0 0 Calcium
13 3 10 4 11 6 1 1 1 0 6 14 0 0 0 0 7 13 0 0 1 0 0 Chloride
1 6 5 10 5 31 2 16 0 0 3 10 0 0 2 0 0 0 0 0 1 0 0 Copper
0 4 4 8 3 40 4 7 0 0 2 5 0 3 2 0 0 0 0 0 8 1 0 Iron
5 1 3 19 4 11 0 2 2 0 0 38 0 1 2 0 0 2 1 0 0 0 0 Iodine
0 2 13 9 5 27 5 1 2 0 2 17 0 3 1 0 0 0 0 0 3 1 0 Magnesium
0 1 0 8 2 18 7 0 0 0 0 52 0 0 1 0 0 0 0 1 2 0 0 Manganese
0 6 4 6 4 47 5 6 6 0 2 5 0 0 1 0 0 0 0 0 0 0 0 Phosphorus
0 6 16 13 10 24 1 1 2 0 5 14 0 0 0 0 0 0 0 0 1 0 0 Potassium
5 1 2 10 5 10 5 5 0 2 1 38

In [401]:
# Minimize energy, but now remove lower bound of energy.
# Reset the model and run the cells for the objective function, decision variables,
# big M constraint, and fat and fatty acid constraints, then go to this 
# cell and create the first model.
# Create constraints for nutrients
model.addConstr(gp.quicksum(x[i]*tables[i][4] for i in range(len(tables))) <= 2874.5) # energy
model.addConstr(gp.quicksum(x[i]*tables[i][5] for i in range(len(tables))) <= 6000) # water
model.addConstr(gp.quicksum(x[i]*tables[i][5] for i in range(len(tables))) >= 3020)
model.addConstr(gp.quicksum(x[i]*tables[i][6] for i in range(len(tables))) <= 250) # protein
model.addConstr(gp.quicksum(x[i]*tables[i][6] for i in range(len(tables))) >= 62.8)
model.addConstr(gp.quicksum(x[i]*tables[i][7] for i in range(len(tables))) <= 315) # carbohydrate
model.addConstr(gp.quicksum(x[i]*tables[i][7] for i in range(len(tables))) >= 279.11)
model.addConstr(gp.quicksum(x[i]*tables[i][12] for i in range(len(tables))) <= 2500) # calcium
model.addConstr(gp.quicksum(x[i]*tables[i][12] for i in range(len(tables))) >= 930)
model.addConstr(gp.quicksum(x[i]*tables[i][13] for i in range(len(tables))) <= 3600) # chloride
model.addConstr(gp.quicksum(x[i]*tables[i][13] for i in range(len(tables))) >= 2057.14)
model.addConstr(gp.quicksum(x[i]*tables[i][14] for i in range(len(tables))) <= 5) # copper
model.addConstr(gp.quicksum(x[i]*tables[i][14] for i in range(len(tables))) >= 1.11)
model.addConstr(gp.quicksum(x[i]*tables[i][15] for i in range(len(tables))) <= 45) # iron
model.addConstr(gp.quicksum(x[i]*tables[i][15] for i in range(len(tables))) >= 9.87)
model.addConstr(gp.quicksum(x[i]*tables[i][16] for i in range(len(tables))) <= 600) # iodine
model.addConstr(gp.quicksum(x[i]*tables[i][16] for i in range(len(tables))) >= 154)
model.addConstr(gp.quicksum(x[i]*tables[i][17] for i in range(len(tables))) <= 704) # magnesium
model.addConstr(gp.quicksum(x[i]*tables[i][17] for i in range(len(tables))) >= 354)
model.addConstr(gp.quicksum(x[i]*tables[i][18] for i in range(len(tables))) <= 11) # manganese
model.addConstr(gp.quicksum(x[i]*tables[i][18] for i in range(len(tables))) >= 3.26)
model.addConstr(gp.quicksum(x[i]*tables[i][19] for i in range(len(tables))) <= 4000) # phosphorus
model.addConstr(gp.quicksum(x[i]*tables[i][19] for i in range(len(tables))) >= 725)
model.addConstr(gp.quicksum(x[i]*tables[i][20] for i in range(len(tables))) <= inf) # potassium
model.addConstr(gp.quicksum(x[i]*tables[i][20] for i in range(len(tables))) >= 3580)
model.addConstr(gp.quicksum(x[i]*tables[i][21] for i in range(len(tables))) <= 300) # selenium
model.addConstr(gp.quicksum(x[i]*tables[i][21] for i in range(len(tables))) >= 63.5)
model.addConstr(gp.quicksum(x[i]*tables[i][22] for i in range(len(tables))) <= 2300) # sodium
model.addConstr(gp.quicksum(x[i]*tables[i][22] for i in range(len(tables))) >= 1702.22)
model.addConstr(gp.quicksum(x[i]*tables[i][23] for i in range(len(tables))) <= 25) # zinc
model.addConstr(gp.quicksum(x[i]*tables[i][23] for i in range(len(tables))) >= 10.9)
model.addConstr(gp.quicksum(x[i]*tables[i][24] for i in range(len(tables))) <= 3000) # vitamin a
model.addConstr(gp.quicksum(x[i]*tables[i][24] for i in range(len(tables))) >= 815)
model.addConstr(gp.quicksum(x[i]*tables[i][25] for i in range(len(tables))) <= 100) # vitamin d
model.addConstr(gp.quicksum(x[i]*tables[i][25] for i in range(len(tables))) >= 11.75)
model.addConstr(gp.quicksum(x[i]*tables[i][26] for i in range(len(tables))) <= 300) # vitamin e
model.addConstr(gp.quicksum(x[i]*tables[i][26] for i in range(len(tables))) >= 11.9)
model.addConstr(gp.quicksum(x[i]*tables[i][27] for i in range(len(tables))) <= inf) # vitamin k
model.addConstr(gp.quicksum(x[i]*tables[i][27] for i in range(len(tables))) >= 92.9)
model.addConstr(gp.quicksum(x[i]*tables[i][28] for i in range(len(tables))) <= 2000) # vitamin c
model.addConstr(gp.quicksum(x[i]*tables[i][28] for i in range(len(tables))) >= 82)
model.addConstr(gp.quicksum(x[i]*tables[i][29] for i in range(len(tables))) <= inf) # vitamin b1
model.addConstr(gp.quicksum(x[i]*tables[i][29] for i in range(len(tables))) >= 1.2)
model.addConstr(gp.quicksum(x[i]*tables[i][30] for i in range(len(tables))) <= inf) # vitamin b2
model.addConstr(gp.quicksum(x[i]*tables[i][30] for i in range(len(tables))) >= 1.415)
model.addConstr(gp.quicksum(x[i]*tables[i][31] for i in range(len(tables))) <= 900) # vitamin b3
model.addConstr(gp.quicksum(x[i]*tables[i][31] for i in range(len(tables))) >= 16.45)
model.addConstr(gp.quicksum(x[i]*tables[i][32] for i in range(len(tables))) <= 200) # vitamin b5
model.addConstr(gp.quicksum(x[i]*tables[i][32] for i in range(len(tables))) >= 5.31)
model.addConstr(gp.quicksum(x[i]*tables[i][33] for i in range(len(tables))) <= 25) # vitamin b6
model.addConstr(gp.quicksum(x[i]*tables[i][33] for i in range(len(tables))) >= 1.51)
model.addConstr(gp.quicksum(x[i]*tables[i][34] for i in range(len(tables))) <= 1000) # vitamin b9
model.addConstr(gp.quicksum(x[i]*tables[i][34] for i in range(len(tables))) >= 315)
model.addConstr(gp.quicksum(x[i]*tables[i][35] for i in range(len(tables))) <= inf) # vitamin b12
model.addConstr(gp.quicksum(x[i]*tables[i][35] for i in range(len(tables))) >= 2.64)
model.addConstr(gp.quicksum(x[i]*tables[i][36] for i in range(len(tables))) <= inf) # fibres
model.addConstr(gp.quicksum(x[i]*tables[i][36] for i in range(len(tables))) >= 30.16)
model.addConstr(gp.quicksum(x[i]*tables[i][37] for i in range(len(tables))) <= 6) # salt
model.addConstr(gp.quicksum(x[i]*tables[i][37] for i in range(len(tables))) >= 0)
model.addConstr(gp.quicksum(x[i]*tables[i][38] for i in range(len(tables))) <= 50) # sugar
model.addConstr(gp.quicksum(x[i]*tables[i][38] for i in range(len(tables))) >= 0)
model.addConstr(gp.quicksum(x[i]*tables[i][39] for i in range(len(tables))) <= 300) # cholesterol
model.addConstr(gp.quicksum(x[i]*tables[i][39] for i in range(len(tables))) >= 0)

<gurobi.Constr *Awaiting Model Update*>

In [422]:
energy = 2000 # a starter value is needed, otherwise we get an error
# The variable energy only appears in the function NutrientValues and in
# this function the current energy amount is taken to calculate the lower
# and upper bounds for fat and the fatty acids
inf = GRB.INFINITY # define infinity to variable 'inf', because some nutrients have no upper bound.
# Change lower bound of energy to 0, because otherwise the function NutrientValues
# would have a lower bound of 1500, with an actual value of 1154.09 which would be illogical
# Also, change the name of Upper_bound to Upper_bound_no_energy
Lower_bound_no_energy = {'Energy': 0, 'Water': 3020, 'Protein': 62.8,
 'Carbohydrate': 279.11, 'Fat': energy*0.2/9, 'Saturated Fat': 0,
 'FA Mono': energy*0.1/9, 'FA Poly': energy*0.03/9, 'Calcium': 930,
 'Chloride': 2057.14, 'Copper': 1.11, 'Iron': 9.87, 'Iodine': 154,
 'Magnesium': 354, 'Manganese': 3.26, 'Phosphorus': 725, 'Potassium': 3580,
 'Selenium': 63.5, 'Sodium': 1702.22, 'Zinc': 10.9, 'Vitamin A': 815,
 'Vitamin D': 11.75, 'Vitamin E': 11.9, 'Vitamin K': 92.9, 'Vitamin C': 82,
 'Vitamin B1': 1.2, 'Vitamin B2': 1.415, 'Vitamin B3': 16.45, 
 'Vitamin B5': 5.31, 'Vitamin B6': 1.51, 'Vitamin B9': 315, 'Vitamin B12': 2.64,
 'Fibres': 30.16, 'Salt': 0, 'Sugar': 0, 'Cholesterol': 0}
Upper_bound_no_energy = {'Energy': 2874.5, 'Water': 6000, 'Protein': 250,
 'Carbohydrate': 315, 'Fat': energy*0.35/9, 'Saturated Fat': energy*0.1/9,
 'FA Mono': energy*0.2/9, 'FA Poly': energy*0.12/9, 'Calcium': 2500,
 'Chloride': 3600, 'Copper': 5, 'Iron': 45, 'Iodine': 600, 'Magnesium': 704,
 'Manganese': 11, 'Phosphorus': 4000, 'Potassium': inf, 'Selenium': 300,
 'Sodium': 2300, 'Zinc': 25, 'Vitamin A': 3000, 'Vitamin D': 100,
 'Vitamin E': 300, 'Vitamin K': inf, 'Vitamin C': 2000, 'Vitamin B1': inf,
 'Vitamin B2': inf, 'Vitamin B3': 900, 'Vitamin B5': 200, 'Vitamin B6': 25,
 'Vitamin B9': 1000, 'Vitamin B12': inf, 'Fibres': inf, 'Salt': 6,
 'Sugar': 50, 'Cholesterol': 300}

In [423]:
# Create function that calculates amount of each nutrient based on diet plan
# This function is altered to change the lower bound of energy to 0
def NutrientValues2():
    nutrient_value_list = [0]*36 # create empty list
    if model.status == GRB.OPTIMAL: # if model is feasible
        for i in range(len(foods)): # loop over every food
            if x[i].x > 0.00001: # if food is in diet plan
                for j in range(len(nutrients)): # then loop over nutrients
                    # multiply every nutrient value by amount in diet and add it to the list
                    nutrient_value_list[j] += df.iloc[i,j+4]*x[i].x
    
    energy = nutrient_value_list[0] # Energy amount in diet plan
    # calculate lower bounds for fat and FA mono/poly
    Lower_bound_no_energy["Fat"] = energy*0.2/9
    Lower_bound_no_energy["FA Mono"] = energy*0.1/9
    Lower_bound_no_energy["FA Poly"] = energy*0.03/9
    lower_bound_list = Lower_bound_no_energy.values() # create list of lower bound values

    # calculate upper bounds for fat, saturated fat and FA mono/poly
    Upper_bound_no_energy["Fat"] = energy*0.35/9
    Upper_bound_no_energy["Saturated Fat"] = energy*0.1/9
    Upper_bound_no_energy["FA Mono"] = energy*0.2/9
    Upper_bound_no_energy["FA Poly"] = energy*0.12/9
    upper_bound_list = Upper_bound_no_energy.values() # create list of upper bound values
    # w = nutrient, x = lb, y = amount in diet plan, z = upper bound
    nutrient_amount = [[w,x,y,z] for w, x, y, z in zip(nutrients, lower_bound_list, nutrient_value_list, upper_bound_list)]
    print(*nutrient_amount, sep = "\n") # every list is printed on a new line


In [404]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3254 rows, 6368 columns and 210885 nonzeros
Model fingerprint: 0x254dff84
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]
Presolve removed 3194 rows and 3191 columns
Presolve time: 0.40s
Presolved: 60 rows, 3177 columns, 176814 nonzeros
Variable types: 3177 continuous, 0 integer (0 binary)

Root relaxation: objective 8.043015e+02, 61 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     804.3014806  804.30148  0.00%     -    0s

Explored 0 nodes (61 simplex iterations) in 0.51 seconds
Thread count was 8 (of 8 avai

In [405]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "seafood, raw") <= 0) # Create a constraint which excludes a food subgroup from the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "beverages") <= 24) # Create a constraint that puts an upper bound on a food group
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tapioca, raw") <= 0) # Constraint that excludes a specific food from the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "offals") <= 0) # Create a constraint which excludes a food subsubgroup from the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "coffee, tea, cocoa beverages, etc. ready to drink") <= 10)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Baking powder or raising agent") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "fish, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Baker's yeast, dehydrated") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Baker's yeast, compressed") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "meat, egg and fish") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "fats and oils") <= 1) # Create an upper bound for the number of fats and oils in the diet plan
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "sauces") <= 0.5)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "vegetables from Martinique and their products") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "fruits from Martinique and their products") <= 0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "meat, egg and fish") <= 4)
for i in range(len(foods)):
    if tables[i][0] == "meat, egg and fish": # If a food from the food group 'meat, egg and fish' is present in the diet plan,
        model.addConstr(x[i] >= 0.5*y[i]) # Then the amount of the food has to be at least 50 g to avoid having small portions
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "seaweed") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "seaweed":
        model.addConstr(x[i] >= 0.05*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "vegetables") <= 5.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "vegetables") <= 5)
for i in range(len(foods)):
    if tables[i][1] == "vegetables":
        model.addConstr(x[i] >= 1.0*y[i])
for i in range(len(foods)):
    if tables[i][1] == "vegetables":
        model.addConstr(x[i] <= 2.5*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "fruits") <= 3.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "fruits") <= 3)
for i in range(len(foods)):
    if tables[i][1] == "fruits":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "nuts and seeds") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "nuts and seeds":
        model.addConstr(x[i] >= 0.30*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "baby food") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "potatoes and other tubers") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "potatoes and other tubers") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "potatoes and other tubers":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "legumes") <= 2.0)        
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "legumes") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "legumes":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "breakfast cereals") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "breakfast cereals":
        model.addConstr(x[i] >= 0.5*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "cooking aids") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "condiments") <= 0.30)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "foods for particular nutritional uses") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "cooking ingredients") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "alcoholic beverages") <= 5)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "beverages") <= 3)
for i in range(len(foods)):
    if tables[i][0] == "beverages":
        model.addConstr(x[i] >= 1.5*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "sweet biscuits") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "sweet biscuits":
        model.addConstr(x[i] >= 0.1*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "pasta, rice and grains") <= 1.50)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "pasta, rice and grains, raw") <= 0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "pasta, rice and grains") <= 2)
for i in range(len(foods)):
    if tables[i][1] == "pasta, rice and grains":
        model.addConstr(x[i] >= 0.75*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "savoury biscuits") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "savoury biscuits":
        model.addConstr(x[i] >= 0.50*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "starters and dishes") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][0] == "starters and dishes") <= 2)
for i in range(len(foods)):
    if tables[i][0] == "starters and dishes":
        model.addConstr(x[i] >= 1.0*y[i])
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "milk") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "milk":
        model.addConstr(x[i] >= 1.5*y[i])        
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "dairy products and similar") <= 2.0)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "dairy products and similar") <= 3)
for i in range(len(foods)):
    if tables[i][1] == "dairy products and similar":
        model.addConstr(x[i] >= 0.50*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][2] == "fromages blanc" or tables[i][1] == "cheese and similar") <= 0.60)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "cheese and similar") <= 1)
for i in range(len(foods)):
    if tables[i][1] == "cheese and similar":
        model.addConstr(x[i] >= 0.30*y[i])
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "herbs" or tables[i][1] == "spices") <= 0.3)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "herbs" or tables[i][1] == "spices") <= 2)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][1] == "raw meat") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][0] == "fats and oils") <= 0.4)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "sauces") <= 2)
model.addConstr(gp.quicksum(y[i] for i in range(len(foods)) if tables[i][1] == "breads and similar") <= 2)
for i in range(len(foods)):
    if tables[i][1] == "breads and similar":
        model.addConstr(x[i] >= 0.4*y[i])

In [406]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6209 rows, 6368 columns and 222709 nonzeros
Model fingerprint: 0x263da6eb
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 6e+03]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R3264 by 74.000000000

Presolve removed 1675 rows and 1449 columns
Presolve time: 0.64s
Presolved: 4534 rows, 4919 columns, 154992 nonzeros
Variable types: 2599 continuous, 2320 integer (2320 binary)

Root relaxation: objective 8.577940e+02, 145 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [407]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Morel, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Frog, leg, grilled/pan-fried") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Aspartame sweetener, in tablets" or tables[i][3] == "Stevia sweetener" 
                            or tables[i][3] == "Aspartame, in powder" or tables[i][3] == "Saccharin sweetener") <= 0.0032)
for i in range(len(foods)):
    if tables[i][1] == "cakes and pastry":
        model.addConstr(x[i] >= 0.3*y[i])

In [408]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6297 rows, 6368 columns and 222885 nonzeros
Model fingerprint: 0xf14cfe24
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1596 rows and 1368 columns
Presolve time: 0.78s
Presolved: 4701 rows, 5000 columns, 155210 nonzeros
Variable types: 2597 continuous, 2403 integer (2403 binary)

Root relaxation: objective 1.029409e+03, 102 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1029.40901    0    9          - 1029.40901      -     -    1s
H    

In [409]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Artichoke base, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Quince, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cod roe, smoked, semi-preserved") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chewing gum, without sugar") <= 0.20)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "cream sauce for pizza base") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tomato sauce for pizza base") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Coriander, fresh") <= 0.05)

<gurobi.Constr *Awaiting Model Update*>

In [410]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6304 rows, 6368 columns and 222891 nonzeros
Model fingerprint: 0xe7efc50d
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1607 rows and 1374 columns
Presolve time: 0.61s
Presolved: 4697 rows, 4994 columns, 154969 nonzeros
Variable types: 2593 continuous, 2401 integer (2401 binary)

Root relaxation: objective 1.068314e+03, 107 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1068.31368    0    9          - 1068.31368      -     -    0s
H    

In [411]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Artichoke base, frozen, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Cod liver oil") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Tarragon, fresh") <= 0.10)

<gurobi.Constr *Awaiting Model Update*>

In [412]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6307 rows, 6368 columns and 222894 nonzeros
Model fingerprint: 0x53fb0132
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1613 rows and 1378 columns
Presolve time: 0.59s
Presolved: 4694 rows, 4990 columns, 154868 nonzeros
Variable types: 2591 continuous, 2399 integer (2399 binary)

Root relaxation: objective 1.096311e+03, 124 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1096.31073    0    8          - 1096.31073      -     -    0s
     

In [413]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Black truffle, raw") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Taro, tuber, cooked") <= 0)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Chervil, fresh") <= 0.10)
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Basil, fresh") <= 0.03)

<gurobi.Constr *Awaiting Model Update*>

In [414]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6311 rows, 6368 columns and 222898 nonzeros
Model fingerprint: 0xa31eee27
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1621 rows and 1382 columns
Presolve time: 0.59s
Presolved: 4690 rows, 4986 columns, 154747 nonzeros
Variable types: 2589 continuous, 2397 integer (2397 binary)

Root relaxation: objective 1.096632e+03, 181 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1096.63175    0    9          - 1096.63175      -     -    0s
H    

In [415]:
model.addConstr(gp.quicksum(x[i] for i in range(len(foods)) if tables[i][3] == "Spinous spider crab, boiled") <= 0)

<gurobi.Constr *Awaiting Model Update*>

In [416]:
model.optimize()
DietPlanSolution()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6312 rows, 6368 columns and 222899 nonzeros
Model fingerprint: 0x138afbf1
Variable types: 3184 continuous, 3184 integer (3184 binary)
Coefficient statistics:
  Matrix range     [7e-06, 5e+05]
  Objective range  [6e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 6e+03]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1624 rows and 1384 columns
Presolve time: 0.77s
Presolved: 4688 rows, 4984 columns, 154683 nonzeros
Variable types: 2588 continuous, 2396 integer (2396 binary)

Root relaxation: objective 1.096632e+03, 181 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1096.63175    0   10          - 1096.63175      -     -    1s
H    

In [424]:
NutrientValues2()

['Energy', 0, 1154.0887757946343, 2874.5]
['Water', 3020, 3020.0, 6000]
['Protein', 62.8, 62.79999999999999, 250]
['Carbohydrate', 279.11, 279.11, 315]
['Fat', 25.646417239880762, 25.655151306984994, 44.88123016979134]
['FA Saturated', 0, 12.823208619940393, 12.823208619940381]
['FA Mono', 12.823208619940381, 19.71379001770919, 25.646417239880762]
['FA Poly', 3.8469625859821144, 11.610599756828135, 15.387850343928458]
['Calcium', 930, 2050.1142759075756, 2500]
['Chloride', 2057.14, 3297.5479544619348, 3600]
['Copper', 1.11, 2.437426757278629, 5]
['Iron', 9.87, 22.74218245701279, 45]
['Iodine', 154, 154.00000000000003, 600]
['Magnesium', 354, 597.5772034964857, 704]
['Manganese', 3.26, 7.382959890543148, 11]
['Phosphorus', 725, 1200.2692401166364, 4000]
['Potassium', 3580, 5169.8738820752305, 1e+100]
['Selenium', 63.5, 160.6293115411885, 300]
['Sodium', 1702.22, 2299.9999999999995, 2300]
['Zinc', 10.9, 10.899999999999997, 25]
['Vitamin A', 815, 1069.0237884375551, 3000]
['Vitamin D', 11

In [337]:
NutrientPercentage()

0 4 13 13 23 3 11 10 0 0 11 0 3 0 0 0 0 0 0 0 0 Energy
4 6 4 0 0 2 1 1 56 6 15 0 0 0 1 0 0 0 0 0 0 Water
0 8 3 14 47 0 7 14 0 0 0 0 0 0 0 0 0 0 0 0 0 Protein
0 1 20 26 17 3 9 1 0 0 12 0 5 0 0 0 0 0 0 0 0 Carbohydrate
2 4 2 1 3 0 23 31 0 1 0 0 0 0 28 0 0 0 0 0 0 Fat
2 1 2 0 1 0 1 13 0 0 0 0 0 67 7 0 0 0 0 0 0 FA Saturated
0 0 0 0 1 0 4 8 0 0 0 0 0 63 20 0 0 0 0 0 0 FA Mono
0 4 1 1 3 1 7 21 0 0 0 0 0 39 17 0 0 0 0 0 0 FA Poly
0 11 0 12 1 0 3 8 19 0 0 0 36 0 0 0 0 1 2 0 0 Calcium
20 6 5 3 2 0 5 11 0 1 2 0 0 0 13 1 21 0 0 0 0 Chloride
0 8 6 17 36 8 9 4 0 2 3 0 0 0 0 0 0 0 0 0 0 Copper
0 33 5 6 29 1 2 7 0 0 0 0 4 0 0 0 0 0 4 0 0 Iron
28 4 3 4 6 2 9 30 0 1 0 0 1 0 3 1 0 0 1 0 0 Iodine
0 18 11 15 13 0 2 3 11 13 0 0 4 0 0 0 1 0 1 0 0 Magnesium
0 36 0 18 17 5 12 1 0 1 0 0 0 0 0 0 0 2 1 1 0 Manganese
0 7 5 25 29 0 7 16 0 0 2 0 0 0 1 0 0 0 0 0 0 Phosphorus
0 20 18 31 15 0 2 4 0 3 1 0 0 0 0 0 0 1 0 0 0 Potassium
0 0 2 4 6 1 6 13 53 4 1 0 3 0 3 0 0 0 0 0 0 Selenium
24 6 0 7 0 0 12 12 0 0 0 0 0 0 13